# Data Gathering & Cleaning

## Purpose
This notebook gathers, cleans, and structures YouTube metadata related to professional tennis from the two most-viewed tennis-related channels - Tennis TV and WTA. It is designed to support accurate player recognition, video categorisation, and dataset preparation for downstream analysis and visualisation in Tableau.

---

## Key Steps
- 📥 **Load raw data** from sources.
  - YouTube API videos published by Tennis TV and WTA channels after 2023-01-01 and before 2025-04-21.
  - Scrap the lists of the ATP and WTA Tournaments of 2023 and 2024.
  - Scrap the lists of the Tennis players' names.
- 🧹 **Clean and normalise** video metadata (titles, tags, timestamps, durations, tournament names, locations, players' names, players' countries).
- 🎾 **Extract player & tournament names** from video titles and playlist titles using a custom matching function that:
  - Matches names of the players and carefully handles ambiguous last names.
  - Reduces false positives (e.g., avoids confusing "Medvedev" names or generic words like "Court").
  - Identifies tournament names.
- 🗂️ **Categorize playlists** into groups:
  - Tournaments
  - Highlights
  - Shorts
  - Netflix
  - Exclusives & Others
  - Dramatic & Funny
- 📊 **Create summary tables**:
  - `player_video_df`: individual player–video rows with views, likes, and comments.
  - `player_video_summary`: aggregate player-level statistics (appearance count, total views, likes, comments, video ids).

---

## Outputs
- ✅ `video_df`: cleaned full dataset of tennis YouTube videos published by Tennis TV and WTA channels after 2023-01-01 and before 2025-04-21.
- ✅ `player_video_df`: expanded player–video relationship table.
- ✅ `players_df`: players' names and their countries.
- ✅ `player_video_summary`: for use as brief and fast analysis.
- ✅ All datasets exported to `.xlsx`.

# **Importing necessary Python libraries**


In [409]:
from googleapiclient.discovery import build
from tqdm import tqdm
import numpy as np  
import pandas as pd
import re
from IPython.display import JSON
from bs4 import BeautifulSoup
from collections import Counter
from collections import defaultdict
from fuzzywuzzy import fuzz
import unicodedata
import urllib.request
import requests
import isodate
import warnings

# **Retrieving Data and Data Preparation**


## **Retrieving Tennis TV channel data from YouTube API**


In [14]:
# Defining API key
api_key = 'YOURAPIKEY'

In [16]:
# Creating a variable to store channels IDs: Tennis TV and WTA
channel_ids = [
    'UCbcxFkd6B9xUU54InHv4Tig',  # Tennis TV
    'UCaBIVVpHjq6j3tSyxwTE-8Q'   # WTA
]


In [18]:
# Geting credentials and create an API client
api_service_name = "youtube"
api_version = "v3"

youtube = build(
    api_service_name, api_version, developerKey=api_key)

### Defining functions for retrieving the data

In [21]:
# Defining a function to get the general channels' stats
def get_channel_stats(youtube, channel_ids):
    all_data = []
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids)
    )
    response = request.execute()
    for item in response['items']:
        data = {
            'channelName': item['snippet']['title'],
            'subscribers': item['statistics']['subscriberCount'],
            'views': item['statistics']['viewCount'],
            'totalVideos': item['statistics']['videoCount'],
            'playlistID': item['contentDetails']['relatedPlaylists']['uploads']
        }
        all_data.append(data)
    return pd.DataFrame(all_data)

 Some information about tournament names is contained in the playlist name, so we also need to retrieve the playlist data.

In [24]:
# Defining a function to get all playlists.
def get_all_playlists(youtube, channel_id):
    playlists = []
    request = youtube.playlists().list(
        part="snippet,contentDetails",
        channelId=channel_id,
        maxResults=50
    )
    response = request.execute()
    playlists.extend(response['items'])

    next_page_token = response.get('nextPageToken')
    while next_page_token:
        request = youtube.playlists().list(
            part="snippet,contentDetails",
            channelId=channel_id,
            maxResults=50,
            pageToken=next_page_token
        )
        response = request.execute()
        playlists.extend(response['items'])
        next_page_token = response.get('nextPageToken')

    playlist_data = [{
        'playlist_id': pl['id'],
        'title': pl['snippet']['title'],
        'videoCount': pl['contentDetails']['itemCount']
    } for pl in playlists]

    return pd.DataFrame(playlist_data)

In [26]:
# Defining a function to get all playlist names which contain tournament name information
def get_playlist_title(youtube, playlist_id):
    request = youtube.playlists().list(
        part="snippet",
        id=playlist_id
    )
    response = request.execute()
    if response['items']:
        return response['items'][0]['snippet']['title']
    return None

In [28]:
# Defining a function to get video ids from channels
def get_video_ids(youtube, playlist_id):
    video_ids = []
    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId=playlist_id,
        maxResults=50
    )
    response = request.execute()
    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])

    next_page_token = response.get('nextPageToken')
    while next_page_token:
        request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        )
        response = request.execute()
        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])
        next_page_token = response.get('nextPageToken')

    return video_ids

In [30]:
# Defining a function to get details of video ids
def get_video_details(youtube, video_ids):
    all_video_info = []

    stats_to_keep = {
        'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
        'statistics': ['viewCount', 'likeCount', 'commentCount'],
        'contentDetails': ['duration', 'definition', 'caption']
    }

    for i in tqdm(range(0, len(video_ids), 50), desc="Fetching video details"):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute()

        if 'items' not in response:
            continue

        for video in response['items']:
            video_info = {'video_id': video['id']}
            for k, v_list in stats_to_keep.items():
                video_info.update({v: video[k].get(v, None) for v in v_list})
            all_video_info.append(video_info)

    return pd.DataFrame(all_video_info)

### Retrieving data from both Tennis TV and WTA channels


In [33]:
# General stats of the channels
channel_stats = get_channel_stats(youtube, channel_ids)
channel_stats

,channelName,subscribers,views,totalVideos,playlistID
0,Tennis TV,1780000,2427962746,11581,UUbcxFkd6B9xUU54InHv4Tig
1,WTA,1660000,519716133,12752,UUaBIVVpHjq6j3tSyxwTE-8Q


In [35]:
# Creating empty variables for data
all_playlist_videos = []
all_video_details = []
all_playlist_titles = {}

In [37]:
# Using for loop to get all the data
for channel_id in channel_ids:
    print(f"\n🎬 Processing channel: {channel_id}")

    # Geting playlists for the channel
    playlists_df = get_all_playlists(youtube, channel_id)
    playlist_ids = playlists_df['playlist_id'].tolist()

    # Geting video IDs for each playlist
    for pid in tqdm(playlist_ids, desc=f"Getting video IDs from playlists for {channel_id}"):
        try:
            vids = get_video_ids(youtube, pid)
            for vid in vids:
                all_playlist_videos.append({
                    'playlist_id': pid,
                    'video_id': vid
                })
        except Exception as e:
            print(f"❌ Error with playlist {pid}: {e}")

    # Storing playlist titles for mapping
    for pid in tqdm(playlist_ids, desc=f"Fetching playlist titles for {channel_id}"):
        try:
            all_playlist_titles[pid] = get_playlist_title(youtube, pid)
        except Exception as e:
            print(f"❌ Error fetching title for playlist {pid}: {e}")


🎬 Processing channel: UCbcxFkd6B9xUU54InHv4Tig


Getting video IDs from playlists for UCbcxFkd6B9xUU54InHv4Tig: 100%|██████████| 560/560 [02:44<00:00,  3.40it/s]
Fetching playlist titles for UCbcxFkd6B9xUU54InHv4Tig: 100%|██████████| 560/560 [01:34<00:00,  5.93it/s]



🎬 Processing channel: UCaBIVVpHjq6j3tSyxwTE-8Q


Getting video IDs from playlists for UCaBIVVpHjq6j3tSyxwTE-8Q: 100%|██████████| 326/326 [01:21<00:00,  4.01it/s]
Fetching playlist titles for UCaBIVVpHjq6j3tSyxwTE-8Q: 100%|██████████| 326/326 [00:51<00:00,  6.32it/s]


In [38]:
# Creating playlist-video mapping DataFrame
playlist_videos_df = pd.DataFrame(all_playlist_videos)

In [39]:
# Fetching video details for all unique video IDs
all_video_ids = playlist_videos_df['video_id'].unique().tolist()
video_df = get_video_details(youtube, all_video_ids)

Fetching video details: 100%|██████████| 359/359 [01:28<00:00,  4.05it/s]


In [40]:
# Adding playlist title to each video
playlist_videos_df['playlist_title'] = playlist_videos_df['playlist_id'].map(all_playlist_titles)

In [41]:
# Using Merge to combine all video info
video_df = video_df.merge(playlist_videos_df, on='video_id', how='left')

In [42]:
# Final checking
print("\n🎉 Combined video dataframe:")
video_df


🎉 Combined video dataframe:


,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,commentCount,duration,definition,caption,playlist_id,playlist_title
0,MSfab0Ujhps,Tennis TV,Francisco Cerundolo vs Jakub Mensik Highlights...,Crazy match 🥵\n\nSUBSCRIBE to our channel for ...,"[Tennis, Sport, Sports, ATP, ATP Tour, Hot Sho...",2025-05-01T15:39:20Z,12193,374,108,PT4M20S,hd,false,PLQHHr8gPOsH5Sm4vVksB-khjOX2tUtTHW,Madrid 2025: 23 April - 4 May
1,6aPpwEavxnQ,Tennis TV,Casper Ruud vs Daniil Medvedev Highlights | Ma...,For a spot in the Madrid semi-finals 🧱\n\nSUBS...,"[Tennis, Sport, Sports, ATP, ATP Tour, Hot Sho...",2025-05-01T13:20:47Z,57021,1425,127,PT4M32S,hd,false,PLQHHr8gPOsH5Sm4vVksB-khjOX2tUtTHW,Madrid 2025: 23 April - 4 May
2,6aPpwEavxnQ,Tennis TV,Casper Ruud vs Daniil Medvedev Highlights | Ma...,For a spot in the Madrid semi-finals 🧱\n\nSUBS...,"[Tennis, Sport, Sports, ATP, ATP Tour, Hot Sho...",2025-05-01T13:20:47Z,57021,1425,127,PT4M32S,hd,false,PLQHHr8gPOsH7bu0i5H1kusyBn4ON5mEkr,The Best of Casper Ruud
3,omy1vCWLbZk,Tennis TV,"Musetti Faces De Minaur, Draper Plays Paul; Di...",Lorenzo Musetti battles Alex de Minaur in the ...,"[Tennis, Sport, Sports, ATP, ATP Tour, Hot Sho...",2025-04-30T23:15:38Z,245768,3212,345,PT12M56S,hd,false,PLQHHr8gPOsH5Sm4vVksB-khjOX2tUtTHW,Madrid 2025: 23 April - 4 May
4,GvmOpGJDLzY,Tennis TV,Marriage Proposal Before Tennis Match! 💍👰🏻‍♀️,"Game, set, marriage!\n\nSUBSCRIBE to our chann...","[Tennis, Sport, Sports, ATP, ATP Tour, Hot Sho...",2025-04-30T21:45:59Z,142947,4329,107,PT33S,hd,false,PLQHHr8gPOsH5Sm4vVksB-khjOX2tUtTHW,Madrid 2025: 23 April - 4 May
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23430,kOYZ27FLphg,WTA,Simona Halep | 2016 Rogers Cup - Pre Tournamen...,Subscribe to WTA on YouTube: http://www.youtub...,"[tennis highlights, tennis, tenis, WTA, Women'...",2016-07-25T17:38:30Z,2579,62,2,PT2M27S,sd,false,PLA06901A4154F49E2,Interviews
23431,07OA_DEyNtE,WTA,Carla Suarez Navarro | 2016 Rogers Cup - Pre T...,Subscribe to WTA on YouTube: http://www.youtub...,"[tennis highlights, tennis, tenis, WTA, Women'...",2016-07-25T17:31:47Z,582,9,0,PT1M56S,sd,false,PLA06901A4154F49E2,Interviews
23432,DsGqh_sSXlo,WTA,Garbine Muguruza | 2016 Rogers Cup - Pre Tourn...,Subscribe to WTA on YouTube: http://www.youtub...,"[tennis highlights, tennis, tenis, WTA, Women'...",2016-07-25T18:03:57Z,2192,22,2,PT1M38S,sd,false,PLA06901A4154F49E2,Interviews
23433,gCD90rGh37Q,WTA,Roberta Vinci | 2016 Rogers Cup - Pre Tourname...,Subscribe to WTA on YouTube: http://www.youtub...,"[tennis highlights, tennis, tenis, WTA, Women'...",2016-07-25T18:07:29Z,1064,16,0,PT2M12S,sd,false,PLA06901A4154F49E2,Interviews


In [49]:
# Displaying unique playlist titles and their counts
playlist_counts = video_df['playlist_title'].value_counts()

print(playlist_counts)


playlist_title
Short Tennis Videos                             1943
Highlights                                       293
Miami                                            237
ATP Tennis Compilations                          228
ATP Tennis Funny Moments & Fails                 210
                                                ... 
Sofia 2023: 6-11 Nov                               1
Best ATP Tennis Matches Of 2025                    1
Match Point Moments | WTA x Morgan Stanley         1
When Passion Meets Performance | Rolex x WTA       1
Santiago 2024: Feb 26th - Mar 2nd                  1
Name: count, Length: 878, dtype: int64


In [51]:
# Getting the number of "Short tennis videos"
short_tennis_videos_count = len(video_df[video_df['playlist_title'] == 'Short Tennis Videos'])
total_videos = len(video_df)

# Calculating the percentage of Short tennis videos of the total number of videos
percentage = (short_tennis_videos_count / total_videos) * 100

print(f"'Short tennis videos' make up {percentage:.2f}% of all videos.")

'Short tennis videos' make up 8.29% of all videos.


## **Data pre-processing**

In [53]:
# Checking null values
video_df.isnull().any()

video_id          False
channelTitle      False
title             False
description       False
tags               True
publishedAt       False
viewCount          True
likeCount          True
commentCount       True
duration          False
definition        False
caption           False
playlist_id       False
playlist_title    False
dtype: bool

### For the purposes of the project it is important to handle all null values in core columns - viewCount, likeCount and commentCount

In [55]:
# Checking how many null values likeCount column has
video_df['likeCount'].isnull().sum()

10

In [57]:
# Exploring those null values
video_df[video_df['likeCount'].isnull()]

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,commentCount,duration,definition,caption,playlist_id,playlist_title
15654,VKYArtzRB8o,WTA,"Semifinals: Sabalenka Faces Gauff, Zheng Again...",All you need to know for Day 7 of action at th...,None,2024-11-08T10:45:11Z,5553,None,17,PT5M12S,hd,false,PLhQBpwasxUpnjMhDblBKnRMK0SWPnU3jj,2024 WTA Finals Riyadh presented by PIF
17487,mTMqXRYf2Hg,WTA,Taking you to Wuhan ✨ Inside The Tour: Wuhan O...,Subscribe to the WTA on YouTube: http://www.yo...,None,2024-10-18T14:27:53Z,48572,None,1,PT22S,hd,false,PLhQBpwasxUplxDRD7g_VBXgkI3esHkC38,#Shorts
17494,uHBA7z_EG7U,WTA,Zheng Qinwen and Aryna Sabalenka serving up a ...,Subscribe to the WTA on YouTube: http://www.yo...,None,2024-10-13T11:55:00Z,79241,None,37,PT32S,hd,false,PLhQBpwasxUplxDRD7g_VBXgkI3esHkC38,#Shorts
23143,h5nqpK0sK5k,WTA,Maria Sharapova: Cincinnati Training,In the build up to the Western & Southern Fina...,"[Tennis, Maria Sharapova, Training, Cincinnati...",2010-08-07T23:23:25Z,378039,None,15,PT52S,hd,false,PL860024DBAC284B94,Health & Fitness Videos
23167,uAQjNjiMFkQ,WTA,Wimbledon Player Party,"Held at the Kensington Roof Gardens, the Sony ...","[Ana, Ivanovic, Maria, Sharapova, Venus, Willi...",2010-06-18T15:29:34Z,22683,None,2,PT35S,sd,false,PLB3D4A15C3FF86C36,"Glamour, Red Carpets & Fashion"
23168,tPoF1SV_HOE,WTA,Pre-Wimbledon Party,The tennis party of the year took place at the...,"[wta, wta tennis, wta tour, wtatour, wta ranki...",2010-06-18T13:49:33Z,17470,None,5,PT2M43S,sd,false,PLB3D4A15C3FF86C36,"Glamour, Red Carpets & Fashion"
23418,KVQZMzq5LgU,WTA,Elena Dementieva At The 2010 Rogers Cup,We caught up with Elena Dementieva ahead of th...,"[Elena Dementieva, Rogers Cup, Montreal, Canad...",2010-08-17T03:29:54Z,3436,None,2,PT2M14S,hd,false,PLA06901A4154F49E2,Interviews
23419,KVQZMzq5LgU,WTA,Elena Dementieva At The 2010 Rogers Cup,We caught up with Elena Dementieva ahead of th...,"[Elena Dementieva, Rogers Cup, Montreal, Canad...",2010-08-17T03:29:54Z,3436,None,2,PT2M14S,hd,false,PLA06901A4154F49E2,Interviews
23424,LN-MVZcQc18,WTA,Agnieszka Radwanska at the Rogers Cup 2010,Agnieszka Radwanska talks to WTA about the Rog...,"[Agnieszka Radwanska, tennis, WTA, ATP, Grand ...",2010-08-17T03:13:06Z,4777,None,2,PT2M48S,hd,false,PLA06901A4154F49E2,Interviews
23425,LN-MVZcQc18,WTA,Agnieszka Radwanska at the Rogers Cup 2010,Agnieszka Radwanska talks to WTA about the Rog...,"[Agnieszka Radwanska, tennis, WTA, ATP, Grand ...",2010-08-17T03:13:06Z,4777,None,2,PT2M48S,hd,false,PLA06901A4154F49E2,Interviews


Concerning a small number of views, we can drop those null values, and it would not affect our analysis that much.

In [61]:
# Dropping null values
video_df = video_df.dropna(subset=['likeCount'])

# Double-checking
video_df['likeCount'].isnull().sum()

0

In [65]:
# Checking how many null values viewCount column has
video_df['viewCount'].isnull().sum()

1

In [74]:
# Exploring that null value
video_df[video_df['viewCount'].isnull()]

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,commentCount,duration,definition,caption,playlist_id,playlist_title
23300,Ft-bhbAlPz0,WTA,Stacey Allaster's State of WTA address,,"[#hangoutsonair, Hangouts On Air, #hoa]",2012-10-28T11:57:13Z,None,15,0,PT22M45S,sd,false,PLF33720ACB6FE8A8F,Istanbul: TEB BNP Paribas WTA Championships


As soon as this video was published in 2012 we can drop it, also since it has only 15 likes we can assume that this video was not popular and has a 
minor amount of views

In [34]:
# Dropping that null value
video_df = video_df.dropna(subset=['viewCount'])

# Double-checking
video_df['viewCount'].isnull().sum()

0

In [72]:
# Checking how many null values commentCount column has
video_df['commentCount'].isnull().sum()

1

In [76]:
# Exploring that null value
video_df[video_df['commentCount'].isnull()]

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,commentCount,duration,definition,caption,playlist_id,playlist_title
22028,WFbIfgijjDY,WTA,WTA Live Fan Access Presented by Xerox | Grand...,WTA on Facebook: http://www.facebook.com/WTA\n...,"[tennis highlights, tennis, tenis, WTA, Women'...",2015-12-15T13:28:47Z,812,17,None,PT2M44S,hd,false,PLhQBpwasxUpluwn05--NOQO4Ve8N_abLr,WTA Live Fan Access presented by Xerox


As soon as video was published in 2015 and has small amount of viewCount this video would not affect our analysis.

In [82]:
# Dropping that null value
video_df = video_df.dropna(subset=['commentCount'])

# Double-checking
video_df['commentCount'].isnull().sum()

0

In [84]:
# Ensuring column formats
numeric_cols = ['viewCount', 'likeCount', 'commentCount']
video_df[numeric_cols] = video_df[numeric_cols].apply(pd.to_numeric, errors = 'coerce', axis = 1)

# Ensuring the 'published_date' column is in datetime format
video_df['publishedAt'] = pd.to_datetime(video_df['publishedAt'])

In [86]:
# Dropping duplicates
video_df = video_df.drop_duplicates(subset='video_id').reset_index(drop=True)

In [88]:
# Converting duration column values into seconds
video_df['durationSeconds'] = video_df['duration'].apply(
    lambda x: isodate.parse_duration(x).total_seconds() if pd.notnull(x) else None
)

In [90]:
# Double-checking
video_df[['durationSeconds', 'duration']]

,durationSeconds,duration
0,260.0,PT4M20S
1,272.0,PT4M32S
2,776.0,PT12M56S
3,33.0,PT33S
4,103.0,PT1M43S
...,...,...
17659,147.0,PT2M27S
17660,116.0,PT1M56S
17661,98.0,PT1M38S
17662,132.0,PT2M12S


In [92]:
# Adding count of tags
video_df['tagCount'] = video_df['tags'].apply(lambda x: 0 if x is None else len(x))

In [94]:
# Filtering videos between Jan 1, 2023 and Apr 21, 2025
filtered_videos = video_df[
    (video_df['publishedAt'] > '2023-01-01') &
    (video_df['publishedAt'] < '2025-04-21')
]

In [96]:
video_df = filtered_videos
print(f"Number of videos: {len(video_df)}")

Number of videos: 6971


# To perform future analysis, it's crucial where it is possible to match videos and it's tournaments.

## As first step, we should retrieve the list of all the major tournaments and define relations between videos and tournaments.

In [105]:
# Setting urls for scraping
url_2023_atp = 'https://www.espn.com/tennis/schedule/_/season/2023'
url_2023_wta = 'https://www.espn.com/tennis/schedule/_/season/2023/type/wta'
url_2024_atp = 'https://www.espn.com/tennis/schedule/_/season/2024'
url_2024_wta = 'https://www.espn.com/tennis/schedule/_/season/2024/type/wta'

In [109]:
# Loading the data
url_2023_atp = 'https://www.espn.com/tennis/schedule/_/season/2023'
list_2023_atp = pd.read_html(url_2023_atp)
list_2023_atp = list_2023_atp[0]

In [111]:
# Defining a function to clean the 'tournament' column
def clean_tournament_name(tournament_str):
    # Use regex to add a space before the location (when there's no space)
    return re.sub(r'([a-z])([A-Z])', r'\1 \2', tournament_str)

list_2023_atp['tournament'] = list_2023_atp['tournament'].apply(clean_tournament_name)

# Defining a function to separate tournament name, city, and country
def separate_tournament_city_country(tournament_str):
    # Pattern to match tournament name followed by city and country
    # ESPN format is typically "TournamentNameCity, Country"
    match = re.match(r'^(.+?)([A-Z][a-z]+)(?:,\s*([A-Za-z\s]+))?$', tournament_str)
    
    if match:
        tournament_name = match.group(1).strip()
        city = match.group(2).strip()
        country = match.group(3).strip() if match.group(3) else ""
        return tournament_name, city, country
    else:
        # Try alternative pattern if the first one fails
        match = re.match(r'^(.+?),\s*([^,]+)$', tournament_str)
        if match:
            tournament_name = match.group(1).strip()
            location = match.group(2).strip()
            # Split location into city and country if possible
            if ' ' in location:
                city, country = location.rsplit(' ', 1)
                return tournament_name, city.strip(), country.strip()
            else:
                return tournament_name, location, ""
        return tournament_str, "", ""  # Fallback

# Applying the separation to create three new columns
list_2023_atp[['tournament_name', 'city', 'country']] = list_2023_atp['tournament'].apply(
    lambda x: pd.Series(separate_tournament_city_country(x))
)

# Displaying the cleaned dataframe
list_2023_atp

,Dates,tournament,winner,tournament_name,city,country
0,Dec 31 - Jan 8,"Adelaide International Adelaide, Australia",Novak Djokovic,Adelaide International,Adelaide,Australia
1,Dec 31 - Jan 7,"Tata Open Maharashtra Pune, India",Tallon Griekspoor,Tata Open Maharashtra,Pune,India
2,Dec 31 - Jan 13,"ASB Classic Auckland, New Zealand",Richard Gasquet,ASB Classic,Auckland,New Zealand
3,Jan 6 - 14,"Adelaide International 2Adelaide, Australia",SoonWoo Kwon,Adelaide International 2,Adelaide,Australia
4,Jan 8 - 29,"Australian Open Melbourne, Australia View Bracket",Novak Djokovic,Australian Open,Melbourne,Australia View Bracket
...,...,...,...,...,...,...
66,Nov 28 - Dec 2,"Next Gen ATP Finals presented by PIFJeddah, Sa...",Hamad Medjedovic,Next Gen ATP Finals presented by PIF,Jeddah,Saudi Arabia
67,Dec 28 - Jan 7,Brisbane International Presented by Evie Brisb...,Grigor Dimitrov,Brisbane International Presented by Evie,Brisbane,Australia
68,Dec 29 - Jan 7,"Tata Open Maharashtra Pune, India",NaN,Tata Open Maharashtra,Pune,India
69,Dec 29 - Jan 12,"ASB Classic Auckland, New Zealand",Alejandro Tabilo,ASB Classic,Auckland,New Zealand


In [113]:
# Setting table with only columns that we will use
list_2023_atp = list_2023_atp[['tournament_name', 'city', 'country', 'winner']]

In [115]:
list_2023_atp

,tournament_name,city,country,winner
0,Adelaide International,Adelaide,Australia,Novak Djokovic
1,Tata Open Maharashtra,Pune,India,Tallon Griekspoor
2,ASB Classic,Auckland,New Zealand,Richard Gasquet
3,Adelaide International 2,Adelaide,Australia,SoonWoo Kwon
4,Australian Open,Melbourne,Australia View Bracket,Novak Djokovic
...,...,...,...,...
66,Next Gen ATP Finals presented by PIF,Jeddah,Saudi Arabia,Hamad Medjedovic
67,Brisbane International Presented by Evie,Brisbane,Australia,Grigor Dimitrov
68,Tata Open Maharashtra,Pune,India,NaN
69,ASB Classic,Auckland,New Zealand,Alejandro Tabilo


In [117]:
# Checking unique values of the country column to define if the separation was performed correctly
unique_countries = list_2023_atp['country'].unique().tolist()
print(unique_countries)

['Australia', 'India', 'New Zealand', 'Australia View Bracket', 'Argentina', 'France', 'USA', 'Netherlands', 'United Arab Emirates', 'Qatar', 'Brazil', 'Chile', 'Mexico', 'Portugal', 'Morocco', 'Monaco', 'Germany', 'Spain', 'Bosnia and Herzegovina', 'Italy', 'Switzerland', 'France View Bracket', 'Great Britain', 'Great Britain View Bracket', 'Sweden', 'Croatia', 'Austria', 'Canada', 'USAView Bracket', 'China PR', 'Kazakhstan', 'Japan', 'Belgium', 'Bulgaria', 'Saudi Arabia', 'China']


In [121]:
# Some values were not separated appropriately, so there is a need to fix them
# Defining replacements
country_replacements = {
    'Australia View Bracket': 'Australia',
    'France View Bracket': 'France',
    'Great Britain View Bracket': 'Great Britain',
    'USAView Bracket': 'USA',
    'China PR': 'China'
}

# Performing replacements
list_2023_atp.loc[:, 'country'] = list_2023_atp['country'].replace(country_replacements)

# Double-checking
print("Unique countries after replacement:")
print(sorted(list_2023_atp['country'].unique().tolist()))

Unique countries after replacement:
['Argentina', 'Australia', 'Austria', 'Belgium', 'Bosnia and Herzegovina', 'Brazil', 'Bulgaria', 'Canada', 'Chile', 'China', 'Croatia', 'France', 'Germany', 'Great Britain', 'India', 'Italy', 'Japan', 'Kazakhstan', 'Mexico', 'Monaco', 'Morocco', 'Netherlands', 'New Zealand', 'Portugal', 'Qatar', 'Saudi Arabia', 'Spain', 'Sweden', 'Switzerland', 'USA', 'United Arab Emirates']


In [123]:
# Checking the unique values of the city column
unique_cities = list_2023_atp['city'].unique().tolist()
print(unique_cities)

['Adelaide', 'Pune', 'Auckland', 'Melbourne', 'Cordoba', 'Montpellier', 'Dallas', 'Rotterdam', 'Beach', 'Aires', 'Dubai', 'Doha', 'Janeiro', 'Marseille', 'Santiago', 'Acapulco', 'Wells', 'Miami', 'Houston', 'Estoril', 'Marrakech', 'Carlo', 'Munich', 'Barcelona', 'Luka', 'Madrid', 'Rome', 'Geneva', 'Lyon', 'Paris', 'Stuttgart', 'Hertogenbosch', 'Halle', 'London', 'Eastbourne', 'Mallorca', 'Bastad', 'Gstaad', 'Newport', 'Umag', 'Atlanta', 'Hamburg', 'Washington', 'Kitzbuhel', 'Cabos', 'Toronto', 'Cincinnati', 'Salem', 'York', 'Chengdu', 'Zhuhai', 'Beijing', 'Astana', 'Shanghai', 'Tokyo', 'Stockholm', 'Antwerp', 'Vienna', 'Basel', 'Metz', 'Sofia', 'Turin', 'Jeddah', 'Brisbane', 'Kong']


In [129]:
# Some values were not separated appropriately, so there is a need to fix them
# Defining replacements
city_replacements = {
    'Aires': 'Buenos Aires',
    'Janeiro': 'Rio de Janeiro',
    'Wells': 'Indian Wells',
    'Carlo': 'Monte-Carlo',
    'Luka': 'Banja Luka',
    'Hertogenbosch': 's-Hertogenbosch',
    'Cabos': 'Los Cabos',
    'Salem': 'Winston-Salem,',
    'York': 'New York',
    'Kong': 'Hong Kong',
    'Beach': 'Delray Beach'
}

# Performing replacements
list_2023_atp.loc[:, 'city'] = list_2023_atp['city'].replace(city_replacements)

# Double-checking
print("Unique cities after replacement:")
print(sorted(list_2023_atp['city'].unique().tolist()))

Unique cities after replacement:
['Acapulco', 'Adelaide', 'Antwerp', 'Astana', 'Atlanta', 'Auckland', 'Banja Luka', 'Barcelona', 'Basel', 'Bastad', 'Beijing', 'Brisbane', 'Buenos Aires', 'Chengdu', 'Cincinnati', 'Cordoba', 'Dallas', 'Delray Beach', 'Doha', 'Dubai', 'Eastbourne', 'Estoril', 'Geneva', 'Gstaad', 'Halle', 'Hamburg', 'Hong Kong', 'Houston', 'Indian Wells', 'Jeddah', 'Kitzbuhel', 'London', 'Los Cabos', 'Lyon', 'Madrid', 'Mallorca', 'Marrakech', 'Marseille', 'Melbourne', 'Metz', 'Miami', 'Monte-Carlo', 'Montpellier', 'Munich', 'New York', 'Newport', 'Paris', 'Pune', 'Rio de Janeiro', 'Rome', 'Rotterdam', 'Santiago', 'Shanghai', 'Sofia', 'Stockholm', 'Stuttgart', 'Tokyo', 'Toronto', 'Turin', 'Umag', 'Vienna', 'Washington', 'Winston-Salem,', 'Zhuhai', 's-Hertogenbosch']


In [131]:
# Checking the unique values of the tournament_name column
unique_tournament_names = list_2023_atp['tournament_name'].unique().tolist()
print(unique_tournament_names)

['Adelaide International', 'Tata Open Maharashtra', 'ASB Classic', 'Adelaide International 2', 'Australian Open', 'Cordoba Open', 'Open Occitanie', 'Dallas Open', 'ABN Amro Open', 'Delray Beach Open Delray', 'IEB+ Argentina Open Buenos', 'Dubai Duty Free Tennis Championships', 'Qatar Exxon Mobil Open', 'Rio Open presented by Claro Rio de', 'Open 13 Provence', 'Movistar Chile Open', 'Abierto Mexicano Telcel presentado por HSBC', 'BNP Paribas Open Indian', 'Miami Open presented by Itau', "Fayez Sarofim & Co. U.S. Men's Clay Court Championship", 'Millennium Estoril Open', 'Grand Prix Hassan II', 'Rolex Monte-Carlo Masters Monte-', 'BMW Open by Bitpanda', 'Barcelona Open Banc Sabadell', 'Srpska Open Banja', 'Mutua Madrid Open', "Internazionali BNL d'Italia", 'Gonet Geneva Open', 'Open Parc Auvergne-Rhone-Alpes Lyon', 'Roland Garros', 'Boss Open', "Libéma Open's-", 'Terra Wortmann Open', 'HSBC Championships', 'Lexus Eastbourne Open', 'Mallorca Championships', 'Wimbledon', 'Nordea Open', 'EF

In [135]:
# Some values were not separated appropriately, so there is a need to fix them
# Defining replacements
tournament_names_replacements = {
    'Delray Beach Open Delray': 'Delray Beach Open',
    'IEB+ Argentina Open Buenos': 'IEB+ Argentina Open',
    'Rio Open presented by Claro Rio de': 'Rio Open presented by Claro',
    'BNP Paribas Open Indian': 'BNP Paribas Open',
    'Rolex Monte-Carlo Masters Monte-': 'Rolex Monte-Carlo Masters',
    "Libéma Open's-": 'Libéma Open', #different quotation marks for avoiding error
    'Plava Laguna Croatia Open Umag': 'Plava Laguna Croatia Open',
    'Mifel Tennis Open by Telcel Oppo Los': 'Mifel Tennis Open by Telcel Oppo',
    'Winston-Salem Open Winston-': 'Winston-Salem Open',
    'Bank of China Hong Kong Tennis Open Hong Kong, Hong': 'Bank of China Hong Kong Tennis Open',
    'US Open New': 'US Open'
}

# Performing replacements
list_2023_atp.loc[:, 'tournament_name'] = list_2023_atp['tournament_name'].replace(tournament_names_replacements)

# Double-checking
print("Unique tournament_names after replacement:")
print(sorted(list_2023_atp['tournament_name'].unique().tolist()))

Unique tournament_names after replacement:
['ABN Amro Open', 'ASB Classic', 'Abierto Mexicano Telcel presentado por HSBC', 'Adelaide International', 'Adelaide International 2', 'Almaty Open', 'Atlanta Open', 'Australian Open', 'BMW Open by Bitpanda', 'BNP Paribas Nordic Open', 'BNP Paribas Open', 'Bank of China Hong Kong Tennis Open', 'Barcelona Open Banc Sabadell', 'Boss Open', 'Brisbane International Presented by Evie', 'Chengdu Open', 'China Open', 'Cincinnati Open', 'Cordoba Open', 'Dallas Open', 'Delray Beach Open', 'Dubai Duty Free Tennis Championships', 'EFG Swiss Open Gstaad', 'Erste Bank Open', 'European Open', "Fayez Sarofim & Co. U.S. Men's Clay Court Championship", 'Generali Open', 'Gonet Geneva Open', 'Grand Prix Hassan II', 'HSBC Championships', 'Hamburg Open', 'Huafa Properties Zhuhai Championships', 'IEB+ Argentina Open', 'Infosys Hall of Fame Open', "Internazionali BNL d'Italia", 'Kinoshita Group Japan Open Tennis Championships', 'Lexus Eastbourne Open', 'Libéma Open',

In [137]:
# Checking if any null values exist
print(list_2023_atp.isnull().sum())

# Total number of missing values
print("Total missing values:", list_2023_atp.isnull().sum().sum())

tournament_name    0
city               0
country            0
winner             2
dtype: int64
Total missing values: 2


As soon as our main point is to retrieve the names of the tournaments we can allow 2 missing values in 'winner' column.
So we may proceed to 2023 WTA tour tournaments list.

In [143]:
# Loading the data
list_2023_wta = pd.read_html(url_2023_wta)
list_2023_wta = list_2023_wta[0]

In [145]:
# Applying a function to clean the 'tournament' column
list_2023_wta['tournament'] = list_2023_wta['tournament'].apply(clean_tournament_name)

# Applying the separation to create three new columns
list_2023_wta[['tournament_name', 'city', 'country']] = list_2023_wta['tournament'].apply(
    lambda x: pd.Series(separate_tournament_city_country(x))
)

# Exploring the cleaned dataframe
list_2023_wta

,Dates,tournament,winner,tournament_name,city,country
0,Dec 31 - Jan 8,"Adelaide International Adelaide, Australia",Aryna Sabalenka,Adelaide International,Adelaide,Australia
1,Dec 31 - Jan 13,"ASB Classic Auckland, New Zealand",Coco Gauff,ASB Classic,Auckland,New Zealand
2,Jan 1 - 8,"Adelaide International 1Adelaide, Australia",NaN,Adelaide International 1,Adelaide,Australia
3,Jan 6 - 13,"Hobart International Hobart, Australia",Lauren Davis,Hobart International,Hobart,Australia
4,Jan 6 - 14,"Adelaide International 2Adelaide, Australia",Belinda Bencic,Adelaide International 2,Adelaide,Australia
...,...,...,...,...,...,...
94,Dec 10 - 17,"Open BLS de Limoges Limoges, France",Cristina Bucsa,Open BLS de Limoges,Limoges,France
95,Dec 28 - Jan 7,Brisbane International Presented by Evie Brisb...,Elena Rybakina,Brisbane International Presented by Evie,Brisbane,Australia
96,Dec 29 - Jan 12,"ASB Classic Auckland, New Zealand",Coco Gauff,ASB Classic,Auckland,New Zealand
97,Dec 30 - Jan 6,"Workday Canberra International Canberra, Austr...",NaN,Workday Canberra International,Canberra,Australia


In [147]:
# Setting table with only columns that we will use
list_2023_wta = list_2023_wta[['tournament_name', 'city', 'country', 'winner']]
list_2023_wta

,tournament_name,city,country,winner
0,Adelaide International,Adelaide,Australia,Aryna Sabalenka
1,ASB Classic,Auckland,New Zealand,Coco Gauff
2,Adelaide International 1,Adelaide,Australia,NaN
3,Hobart International,Hobart,Australia,Lauren Davis
4,Adelaide International 2,Adelaide,Australia,Belinda Bencic
...,...,...,...,...
94,Open BLS de Limoges,Limoges,France,Cristina Bucsa
95,Brisbane International Presented by Evie,Brisbane,Australia,Elena Rybakina
96,ASB Classic,Auckland,New Zealand,Coco Gauff
97,Workday Canberra International,Canberra,Australia,NaN


In [149]:
# Checking unique values of the country column to define if the separation was performed correctly
unique_countries = list_2023_wta['country'].unique().tolist()
print(unique_countries)

['Australia', 'New Zealand', 'Australia View Bracket', 'Thailand', 'France', 'Colombia', 'Austria', 'United Arab Emirates', 'Qatar', '', 'Mexico', 'USA', 'Germany', 'Spain', 'Italy', 'Morocco', 'France View Bracket', 'Croatia', 'Netherlands', 'Great Britain', 'Great Britain View Bracket', 'Sweden', 'Hungary', 'Romania', 'Poland', 'Switzerland', 'Czechia', 'Canada', 'USAView Bracket', 'Japan', 'Slovenia', 'China PR', 'Korea Republic', 'Tunisia', 'Chile', 'Argentina', 'Andorra', 'Uruguay']


In [151]:
# Some values were not separated appropriately, so there is a need to fix them
# Defining replacements
country_replacements = {
    'Australia View Bracket': 'Australia',
    'France View Bracket': 'France',
    'Great Britain View Bracket': 'Great Britain',
    'USAView Bracket': 'USA',
    'China PR': 'China'
}

# Performing replacements
list_2023_wta.loc[:, 'country'] = list_2023_wta['country'].replace(country_replacements)

# Double-checking
print("Unique countries after replacement:")
print(sorted(list_2023_wta['country'].unique().tolist()))

Unique countries after replacement:
['', 'Andorra', 'Argentina', 'Australia', 'Austria', 'Canada', 'Chile', 'China', 'Colombia', 'Croatia', 'Czechia', 'France', 'Germany', 'Great Britain', 'Hungary', 'Italy', 'Japan', 'Korea Republic', 'Mexico', 'Morocco', 'Netherlands', 'New Zealand', 'Poland', 'Qatar', 'Romania', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Thailand', 'Tunisia', 'USA', 'United Arab Emirates', 'Uruguay']


In [153]:
# Checking country values ''
list_2023_wta['country'].value_counts().get('', 0)

5

In [155]:
# Exploring those rows
list_2023_wta[list_2023_wta['country'] == '']

,tournament_name,city,country,winner
13,"Mérida Open Akron Mérida,",Mexico,,Camila Giorgi
18,"San Luis Potosi Open San Luis Potosí,",Mexico,,NaN
21,TEB BNP Paribas Tennis Championship Istanbul I...,Türkiye,,NaN
32,Open Internacional Femení Solgironès La Bisbal...,Spain,,Arantxa Rus
89,"Mundo Tenis Open Florianópolis,",Brazil,,Ajla Tomljanovic


In [157]:
# It appears that we have problems while parsing, and we can deal with it now by hand because of the small number of affected rows
# For specific rows by index
rows_to_update = [13, 18, 21, 32, 89]
list_2023_wta.loc[rows_to_update, 'country'] = list_2023_wta.loc[rows_to_update, 'city']

In [159]:
print(list_2023_wta.loc[rows_to_update])

                                      tournament_name     city  country  \
13                          Mérida Open Akron Mérida,   Mexico   Mexico   
18              San Luis Potosi Open San Luis Potosí,   Mexico   Mexico   
21  TEB BNP Paribas Tennis Championship Istanbul I...  Türkiye  Türkiye   
32  Open Internacional Femení Solgironès La Bisbal...    Spain    Spain   
89                    Mundo Tenis Open Florianópolis,   Brazil   Brazil   

              winner  
13     Camila Giorgi  
18               NaN  
21               NaN  
32       Arantxa Rus  
89  Ajla Tomljanovic  


In [161]:
# Now we need to fix the city column. Names of the cities are stored in tournament_name
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.width', None,
                       'display.max_colwidth', None):
    print(list_2023_wta.loc[rows_to_update])

                                              tournament_name     city  \
13                                  Mérida Open Akron Mérida,   Mexico   
18                      San Luis Potosi Open San Luis Potosí,   Mexico   
21      TEB BNP Paribas Tennis Championship Istanbul Istanbul  Türkiye   
32  Open Internacional Femení Solgironès La Bisbal d'Empordà,    Spain   
89                            Mundo Tenis Open Florianópolis,   Brazil   

    country            winner  
13   Mexico     Camila Giorgi  
18   Mexico               NaN  
21  Türkiye               NaN  
32    Spain       Arantxa Rus  
89   Brazil  Ajla Tomljanovic  


In [163]:
# Updating row by row
list_2023_wta.at[13, 'city'] = 'Mérida'
list_2023_wta.at[18, 'city'] = 'San Luis Potosí'
list_2023_wta.at[21, 'city'] = 'Istanbul'
list_2023_wta.at[32, 'city'] = "La Bisbal d'Empordà" #different quotation marks for avoiding errors
list_2023_wta.at[89, 'city'] = 'Florianópolis'

# Verifying changes
print(list_2023_wta.loc[[13, 18, 21, 32, 89], ['tournament_name', 'city', 'country']].to_string())

                                              tournament_name                 city  country
13                                  Mérida Open Akron Mérida,               Mérida   Mexico
18                      San Luis Potosi Open San Luis Potosí,      San Luis Potosí   Mexico
21      TEB BNP Paribas Tennis Championship Istanbul Istanbul             Istanbul  Türkiye
32  Open Internacional Femení Solgironès La Bisbal d'Empordà,  La Bisbal d'Empordà    Spain
89                            Mundo Tenis Open Florianópolis,        Florianópolis   Brazil


In [165]:
# Verifying unique countries
print("Unique countries after replacement:")
print(sorted(list_2023_wta['country'].unique().tolist()))

Unique countries after replacement:
['Andorra', 'Argentina', 'Australia', 'Austria', 'Brazil', 'Canada', 'Chile', 'China', 'Colombia', 'Croatia', 'Czechia', 'France', 'Germany', 'Great Britain', 'Hungary', 'Italy', 'Japan', 'Korea Republic', 'Mexico', 'Morocco', 'Netherlands', 'New Zealand', 'Poland', 'Qatar', 'Romania', 'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'Thailand', 'Tunisia', 'Türkiye', 'USA', 'United Arab Emirates', 'Uruguay']


In [169]:
# Checking the unique values of the city column
unique_cities = list_2023_wta['city'].unique().tolist()
print(sorted(unique_cities))

['Abu Dhabi', 'Adelaide', 'Aguascalientes', 'Andorra', 'Angers', 'Auckland', 'Austin', 'Bad Homburg', 'Bari', 'Barranquilla', 'Bastad', 'Beijing', 'Berlin', 'Birmingham', 'Bogota', 'Brisbane', 'Bucharest', 'Budapest', 'Buenos Aires', 'Cali', 'Canberra', 'Cancun', 'Charleston', 'Chicago', 'Cincinnati', 'Cleveland', 'Cluj-Napoca', 'Colina', 'Concord', 'Contrexeville', 'Doha', 'Dubai', 'Eastbourne', 'Florence', 'Florianópolis', 'Gaiba', 'Granby', 'Guadalajara', 'Guangzhou', 'Hamburg', 'Hobart', 'Hong Kong', 'Hua Hin', 'Iasi', 'Indian Wells', 'Istanbul', "La Bisbal d'Empordà", 'Lausanne', 'Limoges', 'Linz', 'Ljubljana', 'London', 'Lyon', 'Madrid', 'Makarska', 'Mazowiecki', 'Melbourne', 'Miami', 'Midland', 'Monastir', 'Monterrey', 'Montevideo', 'Mérida', 'Nanchang', 'New York', 'Ningbo', 'Nottingham', 'Osaka', 'Palermo', 'Paris', 'Parma', 'Prague', 'Rabat', 'Reus', 'Rome', 'Rouen', 'Saint-Malo', 'San Diego', 'San Jose', 'San Luis Potosí', 'Seoul', 'Stanford', 'Strasbourg', 'Stuttgart', 'Tam

In [167]:
# Some values were not separated appropriately, so there is a need to fix them
# Defining replacements
city_replacements = {
    'Aires': 'Buenos Aires',
    'Dhabi': 'Abu Dhabi',
    'Hertogenbosch': 's-Hertogenbosch',
    'Hin': 'Hua Hin',
    'Homburg': 'Bad Homburg',
    'Jose': 'San Jose',
    'Kong': 'Hong Kong',
    'Wells': 'Indian Wells',
    'Malo': 'Saint-Malo',
    'York': 'New York',
    'Napoca': 'Cluj-Napoca',
    'Diego': 'San Diego'
}

# Performing replacements
list_2023_wta.loc[:, 'city'] = list_2023_wta['city'].replace(city_replacements)

# Double-checking
print("Unique cities after replacement:")
print(sorted(list_2023_wta['city'].unique().tolist()))

Unique cities after replacement:
['Abu Dhabi', 'Adelaide', 'Aguascalientes', 'Andorra', 'Angers', 'Auckland', 'Austin', 'Bad Homburg', 'Bari', 'Barranquilla', 'Bastad', 'Beijing', 'Berlin', 'Birmingham', 'Bogota', 'Brisbane', 'Bucharest', 'Budapest', 'Buenos Aires', 'Cali', 'Canberra', 'Cancun', 'Charleston', 'Chicago', 'Cincinnati', 'Cleveland', 'Cluj-Napoca', 'Colina', 'Concord', 'Contrexeville', 'Doha', 'Dubai', 'Eastbourne', 'Florence', 'Florianópolis', 'Gaiba', 'Granby', 'Guadalajara', 'Guangzhou', 'Hamburg', 'Hobart', 'Hong Kong', 'Hua Hin', 'Iasi', 'Indian Wells', 'Istanbul', "La Bisbal d'Empordà", 'Lausanne', 'Limoges', 'Linz', 'Ljubljana', 'London', 'Lyon', 'Madrid', 'Makarska', 'Mazowiecki', 'Melbourne', 'Miami', 'Midland', 'Monastir', 'Monterrey', 'Montevideo', 'Mérida', 'Nanchang', 'New York', 'Ningbo', 'Nottingham', 'Osaka', 'Palermo', 'Paris', 'Parma', 'Prague', 'Rabat', 'Reus', 'Rome', 'Rouen', 'Saint-Malo', 'San Diego', 'San Jose', 'San Luis Potosí', 'Seoul', 'Stanford'

In [71]:
# Checking the unique values of the tournament_name column
unique_tournament_names = list_2023_wta['tournament_name'].unique().tolist()
print(sorted(unique_tournament_names))

['ASB Classic', 'ATX Open', 'Abierto GNP Seguros', 'Abierto Tampico', 'Adelaide International', 'Adelaide International 1', 'Adelaide International 2', 'Aguascalientes Open', 'Australian Open', 'BBVA Open Internacional De Valencia', 'BNP Paribas Open Indian', 'BNP Paribas Warsaw Open', 'Bad Homburg Open powered by Solarwatt Bad', 'Barranquilla Open', 'Berlin Tennis Open', 'Brisbane International Presented by Evie', 'Catalonia Open', "Chicago Women's Open", 'China Open', 'Cincinnati Open', 'Copa Colsanitas Zurich presentado por VISA', 'Copa Oster', 'Creand-Andorrà Open', 'Credit One Charleston Open', 'Cymbiotika San Diego Open San', 'Dow Tennis Classic', 'Dubai Duty Free Tennis Championships', 'Firenze Ladies Open', 'Golden Gate Open at Stanford', 'Grand Est Open 88', 'Grand Prix De Son Altesse Royale La Princesse Lalla Meryem', 'Guadalajara Open Akron presented by Santander', 'Guangzhou Open', 'Hamburg Open', 'Hobart International', 'Hungarian Grand Prix', 'IEB+ Argentina Open Buenos',

In [171]:
# Some values were not separated appropriately, so there is a need to fix them
# Defining replacements
tournament_names_replacements = {
    'BNP Paribas Open Indian': 'BNP Paribas Open',
    'Cymbiotika San Diego Open San': 'Cymbiotika San Diego Open',
    'IEB+ Argentina Open Buenos': 'IEB+ Argentina Open',
    "Libéma Open's-": 'Libéma Open', #different quotation marks for avoiding error
    'Mubadala Silicon Valley Classic San': 'Mubadala Silicon Valley Classic',
    'Mundo Tenis Open Florianópolis,': 'Mundo Tenis Open',
    'Mérida Open Akron Mérida,': 'Mérida Open Akron',
    "Open Internacional Femení Solgironès La Bisbal d'Empordà": 'Open Internacional Femení Solgironès', #different quotation marks for avoiding error
    'Polish Open Grodzisk': 'Polish Open',
    'Prudential Hong Kong Tennis Open Hong': 'Prudential Hong Kong Tennis Open',
    'San Luis Potosi Open San Luis Potosí,': 'San Luis Potosi Open',
    'TEB BNP Paribas Tennis Championship Istanbul Istanbul': 'TEB BNP Paribas Tennis Championship Istanbul',
    'Thailand Open Hua': 'Thailand Open',
    'Transylvania Open Cluj-': 'Transylvania Open',
    'US Open New': 'US Open',
    "L'Open 35 de Saint Malo Saint-": "L'Open 35 de Saint Malo" #different quotation marks for avoiding error
}

# Performing replacements
list_2023_wta.loc[:, 'tournament_name'] = list_2023_wta['tournament_name'].replace(tournament_names_replacements)

# Double-checking
print("Unique tournament_names after replacement:")
print(sorted(list_2023_wta['tournament_name'].unique().tolist()))

Unique tournament_names after replacement:
['ASB Classic', 'ATX Open', 'Abierto GNP Seguros', 'Abierto Tampico', 'Adelaide International', 'Adelaide International 1', 'Adelaide International 2', 'Aguascalientes Open', 'Australian Open', 'BBVA Open Internacional De Valencia', 'BNP Paribas Open', 'BNP Paribas Warsaw Open', 'Bad Homburg Open powered by Solarwatt Bad', 'Barranquilla Open', 'Berlin Tennis Open', 'Brisbane International Presented by Evie', 'Catalonia Open', "Chicago Women's Open", 'China Open', 'Cincinnati Open', 'Copa Colsanitas Zurich presentado por VISA', 'Copa Oster', 'Creand-Andorrà Open', 'Credit One Charleston Open', 'Cymbiotika San Diego Open', 'Dow Tennis Classic', 'Dubai Duty Free Tennis Championships', 'Firenze Ladies Open', 'Golden Gate Open at Stanford', 'Grand Est Open 88', 'Grand Prix De Son Altesse Royale La Princesse Lalla Meryem', 'Guadalajara Open Akron presented by Santander', 'Guangzhou Open', 'Hamburg Open', 'Hobart International', 'Hungarian Grand Prix

In [173]:
# Checking if any null values exist
print(list_2023_wta.isnull().sum())

# Total number of missing values
print("Total missing values:", list_2023_wta.isnull().sum().sum())

tournament_name    0
city               0
country            0
winner             9
dtype: int64
Total missing values: 9


As soon as our main point is to retrieve the names of the tournaments we can allow 9 missing values in 'winner' column. So we may proceed to 2024 ATP tour tournaments list

In [176]:
# Loading the data
list_2024_atp = pd.read_html(url_2024_atp)
list_2024_atp = list_2024_atp[0]

# Applying a function to clean the 'tournament' column
list_2024_atp['tournament'] = list_2024_atp['tournament'].apply(clean_tournament_name)

# Applying the separation to create three new columns
list_2024_atp[['tournament_name', 'city', 'country']] = list_2024_atp['tournament'].apply(
    lambda x: pd.Series(separate_tournament_city_country(x))
)

# Exploring the cleaned dataframe
list_2024_atp

,Dates,tournament,winner,tournament_name,city,country
0,Dec 28 - Jan 7,Brisbane International Presented by Evie Brisb...,Grigor Dimitrov,Brisbane International Presented by Evie,Brisbane,Australia
1,Dec 29 - Jan 7,"Tata Open Maharashtra Pune, India",NaN,Tata Open Maharashtra,Pune,India
2,Dec 29 - Jan 12,"ASB Classic Auckland, New Zealand",Alejandro Tabilo,ASB Classic,Auckland,New Zealand
3,Dec 30 - Jan 7,"Bank of China Hong Kong Tennis Open Hong Kong,...",Andrey Rublev,"Bank of China Hong Kong Tennis Open Hong Kong,...",Kong,China
4,Jan 5 - 13,"Adelaide International Adelaide, Australia",Jiri Lehecka,Adelaide International,Adelaide,Australia
...,...,...,...,...,...,...
66,Nov 10 - 17,"Nitto ATP Finals Turin, Italy",Jannik Sinner,Nitto ATP Finals,Turin,Italy
67,Dec 18 - 22,"Next Gen ATP Finals presented by PIFJeddah, Sa...",Joao Fonseca,Next Gen ATP Finals presented by PIF,Jeddah,Saudi Arabia
68,Dec 26 - Jan 5,Brisbane International Presented by Evie Brisb...,Jiri Lehecka,Brisbane International Presented by Evie,Brisbane,Australia
69,Dec 27 - Jan 10,"ASB Classic Auckland, New Zealand",Gael Monfils,ASB Classic,Auckland,New Zealand


In [178]:
# Setting table with only columns that we will use
list_2024_atp = list_2024_atp[['tournament_name', 'city', 'country', 'winner']]
list_2024_atp

,tournament_name,city,country,winner
0,Brisbane International Presented by Evie,Brisbane,Australia,Grigor Dimitrov
1,Tata Open Maharashtra,Pune,India,NaN
2,ASB Classic,Auckland,New Zealand,Alejandro Tabilo
3,"Bank of China Hong Kong Tennis Open Hong Kong,...",Kong,China,Andrey Rublev
4,Adelaide International,Adelaide,Australia,Jiri Lehecka
...,...,...,...,...
66,Nitto ATP Finals,Turin,Italy,Jannik Sinner
67,Next Gen ATP Finals presented by PIF,Jeddah,Saudi Arabia,Joao Fonseca
68,Brisbane International Presented by Evie,Brisbane,Australia,Jiri Lehecka
69,ASB Classic,Auckland,New Zealand,Gael Monfils


In [180]:
# Checking unique values of the country column to define if the separation was performed correctly
unique_countries = list_2024_atp['country'].unique().tolist()
print(sorted(unique_countries))

['', 'Argentina', 'Australia', 'Australia View Bracket', 'Austria', 'Belgium', 'Brazil', 'Canada', 'Chile', 'China', 'China PR', 'Croatia', 'France', 'France View Bracket', 'Germany', 'Great Britain', 'Great Britain View Bracket', 'India', 'Italy', 'Japan', 'Kazakhstan', 'Mexico', 'Monaco', 'Morocco', 'Netherlands', 'New Zealand', 'Portugal', 'Qatar', 'Romania', 'Saudi Arabia', 'Serbia', 'Spain', 'Sweden', 'Switzerland', 'USA', 'USAView Bracket', 'United Arab Emirates']


In [182]:
# Some values were not separated appropriately, so there is a need to fix them
# Defining replacements
country_replacements = {
    'Australia View Bracket': 'Australia',
    'China PR': 'China',
    'France View Bracket': 'France',
    'Great Britain View Bracket': 'Great Britain',
    'USAView Bracket': 'USA',
}

# Performing replacements
list_2024_atp.loc[:, 'country'] = list_2024_atp['country'].replace(country_replacements)

# Double-Checking
print("Unique countries after replacement:")
print(sorted(list_2024_atp['country'].unique().tolist()))

Unique countries after replacement:
['', 'Argentina', 'Australia', 'Austria', 'Belgium', 'Brazil', 'Canada', 'Chile', 'China', 'Croatia', 'France', 'Germany', 'Great Britain', 'India', 'Italy', 'Japan', 'Kazakhstan', 'Mexico', 'Monaco', 'Morocco', 'Netherlands', 'New Zealand', 'Portugal', 'Qatar', 'Romania', 'Saudi Arabia', 'Serbia', 'Spain', 'Sweden', 'Switzerland', 'USA', 'United Arab Emirates']


In [184]:
# Exploring rows where country values are ''
list_2024_atp[list_2024_atp['country'] == '']

,tournament_name,city,country,winner
7,"Cordoba Open Córdoba,",Argentina,,Luciano Darderi
40,"Nordea Open Båstad,",Sweden,,Nuno Borges


In [186]:
# Manually fixing issues
list_2024_atp.at[7, 'country'] = 'Argentina'
list_2024_atp.at[7, 'city'] = 'Córdoba'
list_2024_atp.at[40, 'country'] = 'Sweden'
list_2024_atp.at[40, 'city'] = 'Bastad'

In [188]:
# Verifying changes
print(list_2024_atp.loc[[7, 40], ['tournament_name', 'city', 'country']].to_string())

          tournament_name     city    country
7   Cordoba Open Córdoba,  Córdoba  Argentina
40    Nordea Open Båstad,   Bastad     Sweden


In [190]:
# Double-Checking
unique_cities = list_2024_atp['city'].unique().tolist()
print(sorted(unique_cities))

['Acapulco', 'Adelaide', 'Aires', 'Almaty', 'Antwerp', 'Atlanta', 'Auckland', 'Barcelona', 'Basel', 'Bastad', 'Beach', 'Beijing', 'Belgrade', 'Brisbane', 'Bucharest', 'Cabos', 'Carlo', 'Chengdu', 'Cincinnati', 'Córdoba', 'Dallas', 'Doha', 'Dubai', 'Eastbourne', 'Estoril', 'Geneva', 'Gstaad', 'Halle', 'Hamburg', 'Hangzhou', 'Hertogenbosch', 'Houston', 'Janeiro', 'Jeddah', 'Kitzbuhel', 'Kong', 'London', 'Lyon', 'Madrid', 'Mallorca', 'Marrakech', 'Marseille', 'Melbourne', 'Metz', 'Miami', 'Montpellier', 'Munich', 'Newport', 'Paris', 'Pune', 'Rome', 'Rotterdam', 'Salem', 'Santiago', 'Shanghai', 'Stockholm', 'Stuttgart', 'Tokyo', 'Toronto', 'Turin', 'Umag', 'Vienna', 'Washington', 'Wells', 'York']


In [192]:
# Some values were not separated appropriately, so there is a need to fix them
# Defining replacements
city_replacements = {
    'Aires': 'Buenos Aires',
    'Beach': 'Delray Beach',
    'Carlo': 'Monte-Carlo',
    'Hertogenbosch': 's-Hertogenbosch',
    'Kong': 'Hong Kong',
    'Janeiro': 'Rio de Janeiro',
    'Wells': 'Indian Wells',
    'Cabos': 'Los Cabos',
    'Salem': 'Winston-Salem,',
    'York': 'New York',
    
}

# Performing replacements
list_2024_atp.loc[:, 'city'] = list_2024_atp['city'].replace(city_replacements)

# Double-Checking
print("Unique cities after replacement:")
print(sorted(list_2024_atp['city'].unique().tolist()))

Unique cities after replacement:
['Acapulco', 'Adelaide', 'Almaty', 'Antwerp', 'Atlanta', 'Auckland', 'Barcelona', 'Basel', 'Bastad', 'Beijing', 'Belgrade', 'Brisbane', 'Bucharest', 'Buenos Aires', 'Chengdu', 'Cincinnati', 'Córdoba', 'Dallas', 'Delray Beach', 'Doha', 'Dubai', 'Eastbourne', 'Estoril', 'Geneva', 'Gstaad', 'Halle', 'Hamburg', 'Hangzhou', 'Hong Kong', 'Houston', 'Indian Wells', 'Jeddah', 'Kitzbuhel', 'London', 'Los Cabos', 'Lyon', 'Madrid', 'Mallorca', 'Marrakech', 'Marseille', 'Melbourne', 'Metz', 'Miami', 'Monte-Carlo', 'Montpellier', 'Munich', 'New York', 'Newport', 'Paris', 'Pune', 'Rio de Janeiro', 'Rome', 'Rotterdam', 'Santiago', 'Shanghai', 'Stockholm', 'Stuttgart', 'Tokyo', 'Toronto', 'Turin', 'Umag', 'Vienna', 'Washington', 'Winston-Salem,', 's-Hertogenbosch']


In [194]:
# Checking the unique values of the tournament_name column
unique_tournament_names = list_2024_atp['tournament_name'].unique().tolist()
print(sorted(unique_tournament_names))

['ABN Amro Open', 'ASB Classic', 'Abierto Mexicano Telcel presentado por HSBC', 'Adelaide International', 'Almaty Open', 'Atlanta Open', 'Australian Open', 'BMW Open by Bitpanda', 'BNP Paribas Nordic Open', 'BNP Paribas Open Indian', 'Bank of China Hong Kong Tennis Open Hong Kong, Hong', 'Barcelona Open Banc Sabadell', 'Belgrade Open', 'Boss Open', 'Brisbane International Presented by Evie', 'Chengdu Open', 'China Open', 'Cincinnati Open', 'Cordoba Open Córdoba,', 'Dallas Open', 'Delray Beach Open Delray', 'Dubai Duty Free Tennis Championships', 'EFG Swiss Open Gstaad', 'Erste Bank Open', 'European Open', "Fayez Sarofim & Co. U.S. Men's Clay Court Championship", 'Generali Open', 'Gonet Geneva Open', 'Grand Prix Hassan II', 'HSBC Championships', 'Hamburg Open', 'Hangzhou Open', 'IEB+ Argentina Open Buenos', 'Infosys Hall of Fame Open', "Internazionali BNL d'Italia", 'Kinoshita Group Japan Open Tennis Championships', 'Lexus Eastbourne Open', "Libéma Open's-", 'Mallorca Championships', 'M

In [196]:
# Some values were not separated appropriately, so there is a need to fix them
# Defining replacements
tournament_names_replacements = {
    'BNP Paribas Open Indian': 'BNP Paribas Open',
    'Bank of China Hong Kong Tennis Open Hong Kong, Hong': 'Bank of China Hong Kong Tennis Open',
    'Cordoba Open Córdoba,': 'Cordoba Open',
    'Delray Beach Open Delray': 'Delray Beach Open',
    'IEB+ Argentina Open Buenos': 'IEB+ Argentina Open',
    "Libéma Open's-": 'Libéma Open', #different quotation marks for avoiding error
    'Mifel Tennis Open by Telcel Oppo Los': 'Mifel Tennis Open by Telcel Oppo',
    'Nordea Open Båstad,': 'Nordea Open',    
    'Rio Open presented by Claro Rio de': 'Rio Open presented by Claro',
    'Rolex Monte-Carlo Masters Monte-': 'Rolex Monte-Carlo Masters',
    'US Open New': 'US Open',
    'Winston-Salem Open Winston-': 'Winston-Salem Open',
}

# Performing replacements
list_2024_atp.loc[:, 'tournament_name'] = list_2024_atp['tournament_name'].replace(tournament_names_replacements)

# Double-Checking
print("Unique tournament_names after replacement:")
print(sorted(list_2024_atp['tournament_name'].unique().tolist()))

Unique tournament_names after replacement:
['ABN Amro Open', 'ASB Classic', 'Abierto Mexicano Telcel presentado por HSBC', 'Adelaide International', 'Almaty Open', 'Atlanta Open', 'Australian Open', 'BMW Open by Bitpanda', 'BNP Paribas Nordic Open', 'BNP Paribas Open', 'Bank of China Hong Kong Tennis Open', 'Barcelona Open Banc Sabadell', 'Belgrade Open', 'Boss Open', 'Brisbane International Presented by Evie', 'Chengdu Open', 'China Open', 'Cincinnati Open', 'Cordoba Open', 'Dallas Open', 'Delray Beach Open', 'Dubai Duty Free Tennis Championships', 'EFG Swiss Open Gstaad', 'Erste Bank Open', 'European Open', "Fayez Sarofim & Co. U.S. Men's Clay Court Championship", 'Generali Open', 'Gonet Geneva Open', 'Grand Prix Hassan II', 'HSBC Championships', 'Hamburg Open', 'Hangzhou Open', 'IEB+ Argentina Open', 'Infosys Hall of Fame Open', "Internazionali BNL d'Italia", 'Kinoshita Group Japan Open Tennis Championships', 'Lexus Eastbourne Open', 'Libéma Open', 'Mallorca Championships', 'Miami O

In [198]:
# Checking if any null values exist
print(list_2024_atp.isnull().sum())

# Total number of missing values
print("Total missing values:", list_2024_atp.isnull().sum().sum())

tournament_name    0
city               0
country            0
winner             2
dtype: int64
Total missing values: 2


As soon as our main point is to retrieve the names of the tournaments we can allow 2 missing values in 'winner' column. So we may proceed to 2023 WTA tour tournaments list

In [201]:
# Loading the data
list_2024_wta = pd.read_html(url_2024_wta)
list_2024_wta = list_2024_wta[0]

# Applying a function to clean the 'tournament' column
list_2024_wta['tournament'] = list_2024_wta['tournament'].apply(clean_tournament_name)

# Applying the separation to create three new columns
list_2024_wta[['tournament_name', 'city', 'country']] = list_2024_wta['tournament'].apply(
    lambda x: pd.Series(separate_tournament_city_country(x))
)

# Exploring the cleaned dataframe
list_2024_wta

,Dates,tournament,winner,tournament_name,city,country
0,Dec 28 - Jan 7,Brisbane International Presented by Evie Brisb...,Elena Rybakina,Brisbane International Presented by Evie,Brisbane,Australia
1,Dec 29 - Jan 12,"ASB Classic Auckland, New Zealand",Coco Gauff,ASB Classic,Auckland,New Zealand
2,Dec 30 - Jan 6,"Workday Canberra International Canberra, Austr...",NaN,Workday Canberra International,Canberra,Australia
3,Dec 30 - Jan 5,"Workday Canberra International Canberra, Austr...",Nuria Parrizas Diaz,Workday Canberra International,Canberra,Australia
4,Jan 5 - 12,"Hobart International Hobart, Australia",Emma Navarro,Hobart International,Hobart,Australia
...,...,...,...,...,...,...
91,Dec 1 - 8,"Mundo Tenis Open Florianópolis, Brazil",Maja Chwalinska,"Mundo Tenis Open Florianópolis,",Brazil,
92,Dec 8 - 15,"Open BLS de Limoges Limoges, France",Viktorija Golubic,Open BLS de Limoges,Limoges,France
93,Dec 26 - Jan 5,Brisbane International Presented by Evie Brisb...,Aryna Sabalenka,Brisbane International Presented by Evie,Brisbane,Australia
94,Dec 27 - Jan 10,"ASB Classic Auckland, New Zealand",Clara Tauson,ASB Classic,Auckland,New Zealand


In [203]:
# Setting table with only columns that we will use
list_2024_wta = list_2024_wta[['tournament_name', 'city', 'country', 'winner']]
list_2024_wta

,tournament_name,city,country,winner
0,Brisbane International Presented by Evie,Brisbane,Australia,Elena Rybakina
1,ASB Classic,Auckland,New Zealand,Coco Gauff
2,Workday Canberra International,Canberra,Australia,NaN
3,Workday Canberra International,Canberra,Australia,Nuria Parrizas Diaz
4,Hobart International,Hobart,Australia,Emma Navarro
...,...,...,...,...
91,"Mundo Tenis Open Florianópolis,",Brazil,,Maja Chwalinska
92,Open BLS de Limoges,Limoges,France,Viktorija Golubic
93,Brisbane International Presented by Evie,Brisbane,Australia,Aryna Sabalenka
94,ASB Classic,Auckland,New Zealand,Clara Tauson


In [205]:
# Checking unique values of the country column to define if the separation was performed correctly
unique_countries = list_2024_wta['country'].unique().tolist()
print(sorted(unique_countries))

['', 'Argentina', 'Australia', 'Australia View Bracket', 'Austria', 'Bolivia', 'Canada', 'Chile', 'China', 'China PR', 'Colombia', 'Croatia', 'Czechia', 'France', 'France View Bracket', 'Germany', 'Great Britain', 'Great Britain View Bracket', 'Hungary', 'India', 'Italy', 'Japan', 'Korea Republic', 'Mexico', 'Morocco', 'Netherlands', 'New Zealand', 'Poland', 'Portugal', 'Qatar', 'Romania', 'Saudi Arabia', 'Slovenia', 'Spain', 'Switzerland', 'Thailand', 'Tunisia', 'USA', 'USAView Bracket', 'United Arab Emirates']


In [207]:
# Some values were not separated appropriately, so there is a need to fix them
# Defining replacements
country_replacements = {
    'Australia View Bracket': 'Australia',
    'France View Bracket': 'France',
    'Great Britain View Bracket': 'Great Britain',
    'USAView Bracket': 'USA',
    'China PR': 'China'
}

# Performing replacements
list_2024_wta.loc[:, 'country'] = list_2024_wta['country'].replace(country_replacements)

# Double-Checking
print("Unique countries after replacement:")
print(sorted(list_2024_wta['country'].unique().tolist()))

Unique countries after replacement:
['', 'Argentina', 'Australia', 'Austria', 'Bolivia', 'Canada', 'Chile', 'China', 'Colombia', 'Croatia', 'Czechia', 'France', 'Germany', 'Great Britain', 'Hungary', 'India', 'Italy', 'Japan', 'Korea Republic', 'Mexico', 'Morocco', 'Netherlands', 'New Zealand', 'Poland', 'Portugal', 'Qatar', 'Romania', 'Saudi Arabia', 'Slovenia', 'Spain', 'Switzerland', 'Thailand', 'Tunisia', 'USA', 'United Arab Emirates']


In [209]:
# Checking country values ''
list_2024_wta['country'].value_counts().get('', 0)

6

In [211]:
# Exploring those rows
list_2024_wta[list_2024_wta['country'] == '']

,tournament_name,city,country,winner
20,Megasaray Hotels Open 2Antalya,Türkiye,,Jessica Bouzas Maneiro
21,"Banorte Tennis Open San Luis Potosí,",Mexico,,Nadia Podoroska
24,Open Internacional Femení Solgironès La Bisbal...,Spain,,Maria Lourdes Carle
49,"Nordea Open Båstad,",Sweden,,Martina Trevisan
82,"Mérida Open Akron Mérida,",Mexico,,Zeynep Sonmez
91,"Mundo Tenis Open Florianópolis,",Brazil,,Maja Chwalinska


In [213]:
# It appears that we have problems while parsing, and we can deal with it now by hand because of the small number of affected rows
# For specific rows by index
rows_to_update = [20, 21, 24, 49, 82, 91]
list_2024_wta.loc[rows_to_update, 'country'] = list_2024_wta.loc[rows_to_update, 'city']

In [215]:
print(list_2024_wta.loc[rows_to_update])

                                      tournament_name     city  country  \
20                     Megasaray Hotels Open 2Antalya  Türkiye  Türkiye   
21               Banorte Tennis Open San Luis Potosí,   Mexico   Mexico   
24  Open Internacional Femení Solgironès La Bisbal...    Spain    Spain   
49                                Nordea Open Båstad,   Sweden   Sweden   
82                          Mérida Open Akron Mérida,   Mexico   Mexico   
91                    Mundo Tenis Open Florianópolis,   Brazil   Brazil   

                    winner  
20  Jessica Bouzas Maneiro  
21         Nadia Podoroska  
24     Maria Lourdes Carle  
49        Martina Trevisan  
82           Zeynep Sonmez  
91         Maja Chwalinska  


In [217]:
# Now we need to fix the city column. Names of the cities are stored in tournament_name
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.width', None,
                       'display.max_colwidth', None):
    print(list_2024_wta.loc[rows_to_update])

                                              tournament_name     city  \
20                             Megasaray Hotels Open 2Antalya  Türkiye   
21                       Banorte Tennis Open San Luis Potosí,   Mexico   
24  Open Internacional Femení Solgironès La Bisbal d'Empordà,    Spain   
49                                        Nordea Open Båstad,   Sweden   
82                                  Mérida Open Akron Mérida,   Mexico   
91                            Mundo Tenis Open Florianópolis,   Brazil   

    country                  winner  
20  Türkiye  Jessica Bouzas Maneiro  
21   Mexico         Nadia Podoroska  
24    Spain     Maria Lourdes Carle  
49   Sweden        Martina Trevisan  
82   Mexico           Zeynep Sonmez  
91   Brazil         Maja Chwalinska  


In [219]:
# Updating row by row
list_2024_wta.at[20, 'city'] = 'Antalya'
list_2024_wta.at[21, 'city'] = 'San Luis Potosí'
list_2024_wta.at[24, 'city'] = "La Bisbal d'Empordà" #different quotation marks for avoiding errors
list_2024_wta.at[49, 'city'] = 'Bastad'
list_2024_wta.at[82, 'city'] = 'Mérida'
list_2024_wta.at[91, 'city'] = 'Florianópolis'

# Verifying changes
print(list_2024_wta.loc[[20, 21, 24, 49, 82, 91], ['tournament_name', 'city', 'country']].to_string())

                                              tournament_name                 city  country
20                             Megasaray Hotels Open 2Antalya              Antalya  Türkiye
21                       Banorte Tennis Open San Luis Potosí,      San Luis Potosí   Mexico
24  Open Internacional Femení Solgironès La Bisbal d'Empordà,  La Bisbal d'Empordà    Spain
49                                        Nordea Open Båstad,               Bastad   Sweden
82                                  Mérida Open Akron Mérida,               Mérida   Mexico
91                            Mundo Tenis Open Florianópolis,        Florianópolis   Brazil


In [221]:
# Check the unique values of the city column
unique_cities = list_2024_wta['city'].unique().tolist()
print(sorted(unique_cities))

['Adelaide', 'Aires', 'Angers', 'Antalya', 'Auckland', 'Austin', 'Bari', 'Barranquilla', 'Bastad', 'Beijing', 'Berlin', 'Birmingham', 'Bogota', 'Brisbane', 'Bucharest', 'Budapest', 'Cali', 'Canberra', 'Charleston', 'Cincinnati', 'Cleveland', 'Colina', 'Contrexeville', 'Cruz', 'Dhabi', 'Diego', 'Doha', 'Dubai', 'Eastbourne', 'Florianópolis', 'Gaiba', 'Guadalajara', 'Guangzhou', 'Hamburg', 'Hertogenbosch', 'Hin', 'Hobart', 'Homburg', 'Iasi', 'Kong', "La Bisbal d'Empordà", 'Limoges', 'Linz', 'Ljubljana', 'Lleida', 'London', 'Madrid', 'Makarska', 'Malo', 'Melbourne', 'Miami', 'Midland', 'Monastir', 'Monterrey', 'Montreux', 'Mumbai', 'Mérida', 'Nanchang', 'Napoca', 'Ningbo', 'Nottingham', 'Oeiras', 'Osaka', 'Palermo', 'Paris', 'Parma', 'Prague', 'Rabat', 'Riyadh', 'Rome', 'Rouen', 'San Luis Potosí', 'Seoul', 'Strasbourg', 'Stuttgart', 'Tampico', 'Tokyo', 'Toronto', 'Valencia', 'Vallarta', 'Warsaw', 'Washington', 'Wells', 'Wuhan', 'York']


In [223]:
# Some values were not separated appropriately, so there is a need to fix them
# Defining replacements
city_replacements = {
    'Aires': 'Buenos Aires',
    'Dhabi': 'Abu Dhabi',
    'Hertogenbosch': 's-Hertogenbosch',
    'Hin': 'Hua Hin',
    'Homburg': 'Bad Homburg',
    'Jose': 'San Jose',
    'Kong': 'Hong Kong',
    'Wells': 'Indian Wells',
    'Malo': 'Saint-Malo',
    'York': 'New York',
    'Napoca': 'Cluj-Napoca',
    'Diego': 'San Diego',
    'Vallarta': 'Nuevo Vallarta'
}

# Performing replacements 
list_2024_wta.loc[:, 'city'] = list_2024_wta['city'].replace(city_replacements)

# Double-Checking
print("Unique cities after replacement:")
print(sorted(list_2024_wta['city'].unique().tolist()))

Unique cities after replacement:
['Abu Dhabi', 'Adelaide', 'Angers', 'Antalya', 'Auckland', 'Austin', 'Bad Homburg', 'Bari', 'Barranquilla', 'Bastad', 'Beijing', 'Berlin', 'Birmingham', 'Bogota', 'Brisbane', 'Bucharest', 'Budapest', 'Buenos Aires', 'Cali', 'Canberra', 'Charleston', 'Cincinnati', 'Cleveland', 'Cluj-Napoca', 'Colina', 'Contrexeville', 'Cruz', 'Doha', 'Dubai', 'Eastbourne', 'Florianópolis', 'Gaiba', 'Guadalajara', 'Guangzhou', 'Hamburg', 'Hobart', 'Hong Kong', 'Hua Hin', 'Iasi', 'Indian Wells', "La Bisbal d'Empordà", 'Limoges', 'Linz', 'Ljubljana', 'Lleida', 'London', 'Madrid', 'Makarska', 'Melbourne', 'Miami', 'Midland', 'Monastir', 'Monterrey', 'Montreux', 'Mumbai', 'Mérida', 'Nanchang', 'New York', 'Ningbo', 'Nottingham', 'Nuevo Vallarta', 'Oeiras', 'Osaka', 'Palermo', 'Paris', 'Parma', 'Prague', 'Rabat', 'Riyadh', 'Rome', 'Rouen', 'Saint-Malo', 'San Diego', 'San Luis Potosí', 'Seoul', 'Strasbourg', 'Stuttgart', 'Tampico', 'Tokyo', 'Toronto', 'Valencia', 'Warsaw', 'Was

In [225]:
# Checking the unique values of the tournament_name column
unique_tournament_names = list_2024_wta['tournament_name'].unique().tolist()
print(sorted(unique_tournament_names))

['ASB Classic', 'ATX Open', 'Abierto GNP Seguros', 'Abierto Tampico', 'Adelaide International', 'Australian Open', 'BBVA Open Internacional De Valencia', 'BNP Paribas Open Indian', 'Bad Homburg Open powered by Solarwatt Bad', 'Banorte Tennis Open San Luis Potosí,', 'Barranquilla Open', 'Berlin Tennis Open', 'Bolivia Open Santa', 'Brisbane International Presented by Evie', 'Cali Open', 'Catalonia Open', 'China Open', 'Cincinnati Open', 'Copa Colsanitas Zurich presentado por VISA', 'Credit One Charleston Open', 'Cymbiotika San Diego Open San', 'Dongfeng Voyah Wuhan Open', 'Dow Tennis Classic', 'Dubai Duty Free Tennis Championships', 'Fifth Third Charleston 125', 'Fifth Third Charleston 125 2', 'Grand Est Open 88', 'Grand Prix De Son Altesse Royale La Princesse Lalla Meryem', 'Guadalajara 125 Open', 'Guadalajara Open Akron presented by Santander', 'Guangzhou Open', 'Hamburg European Open', 'Hobart International', 'Hong Kong 125 Open Hong Kong, Hong', 'Hungarian Grand Prix', 'IEB+ Argentin

In [227]:
# Some values were not separated appropriately, so there is a need to fix them
# Defining replacements
tournament_names_replacements = {
    'BNP Paribas Open Indian': 'BNP Paribas Open',
    'Banorte Tennis Open San Luis Potosí,': 'Banorte Tennis Open',
    'Cymbiotika San Diego Open San': 'Cymbiotika San Diego Open',
    'IEB+ Argentina Open Buenos': 'IEB+ Argentina Open',
    'Hong Kong 125 Open Hong Kong, Hong': 'Hong Kong 125 Open',
    "Libéma Open's-": 'Libéma Open', #different quotation marks for avoiding error
    "L'Open 35 de Saint Malo Saint-" : "L'Open 35 de Saint Malo", #different quotation marks for avoiding error
    'Megasaray Hotels Open 2Antalya': 'Megasaray Hotels Open 2',
    'Mubadala Silicon Valley Classic San': 'Mubadala Silicon Valley Classic',
    'Mundo Tenis Open Florianópolis,': 'Mundo Tenis Open',
    'Mérida Open Akron Mérida,': 'Mérida Open Akron',
    'Nordea Open Båstad,': 'Nordea Open',
    "Open Internacional Femení Solgironès La Bisbal d'Empordà": 'Open Internacional Femení Solgironès', #different quotation marks for avoiding error
    'Polish Open Grodzisk': 'Polish Open',
    'Prudential Hong Kong Tennis Open Hong Kong, Hong': 'Prudential Hong Kong Tennis Open',
    'Puerto Vallarta Open Nuevo': 'Puerto Vallarta Open',
    'San Luis Potosi Open San Luis Potosí,': 'San Luis Potosi Open',
    'TEB BNP Paribas Tennis Championship Istanbul Istanbul': 'TEB BNP Paribas Tennis Championship Istanbul',
    'Thailand Open 2Hua': 'Thailand Open',
    'Transylvania Open Cluj-': 'Transylvania Open',
    'US Open New': 'US Open'
}

# Performing replacements
list_2024_wta.loc[:, 'tournament_name'] = list_2024_wta['tournament_name'].replace(tournament_names_replacements)

# Double-Checking
print("Unique tournament_names after replacement:")
print(sorted(list_2024_wta['tournament_name'].unique().tolist()))

Unique tournament_names after replacement:
['ASB Classic', 'ATX Open', 'Abierto GNP Seguros', 'Abierto Tampico', 'Adelaide International', 'Australian Open', 'BBVA Open Internacional De Valencia', 'BNP Paribas Open', 'Bad Homburg Open powered by Solarwatt Bad', 'Banorte Tennis Open', 'Barranquilla Open', 'Berlin Tennis Open', 'Bolivia Open Santa', 'Brisbane International Presented by Evie', 'Cali Open', 'Catalonia Open', 'China Open', 'Cincinnati Open', 'Copa Colsanitas Zurich presentado por VISA', 'Credit One Charleston Open', 'Cymbiotika San Diego Open', 'Dongfeng Voyah Wuhan Open', 'Dow Tennis Classic', 'Dubai Duty Free Tennis Championships', 'Fifth Third Charleston 125', 'Fifth Third Charleston 125 2', 'Grand Est Open 88', 'Grand Prix De Son Altesse Royale La Princesse Lalla Meryem', 'Guadalajara 125 Open', 'Guadalajara Open Akron presented by Santander', 'Guangzhou Open', 'Hamburg European Open', 'Hobart International', 'Hong Kong 125 Open', 'Hungarian Grand Prix', 'IEB+ Argentina

In [229]:
# Checking the unique values of the tournament_name column
unique_tournament_names = list_2024_wta['tournament_name'].unique().tolist()
print(sorted(unique_tournament_names))

['ASB Classic', 'ATX Open', 'Abierto GNP Seguros', 'Abierto Tampico', 'Adelaide International', 'Australian Open', 'BBVA Open Internacional De Valencia', 'BNP Paribas Open', 'Bad Homburg Open powered by Solarwatt Bad', 'Banorte Tennis Open', 'Barranquilla Open', 'Berlin Tennis Open', 'Bolivia Open Santa', 'Brisbane International Presented by Evie', 'Cali Open', 'Catalonia Open', 'China Open', 'Cincinnati Open', 'Copa Colsanitas Zurich presentado por VISA', 'Credit One Charleston Open', 'Cymbiotika San Diego Open', 'Dongfeng Voyah Wuhan Open', 'Dow Tennis Classic', 'Dubai Duty Free Tennis Championships', 'Fifth Third Charleston 125', 'Fifth Third Charleston 125 2', 'Grand Est Open 88', 'Grand Prix De Son Altesse Royale La Princesse Lalla Meryem', 'Guadalajara 125 Open', 'Guadalajara Open Akron presented by Santander', 'Guangzhou Open', 'Hamburg European Open', 'Hobart International', 'Hong Kong 125 Open', 'Hungarian Grand Prix', 'IEB+ Argentina Open', 'Internationaux de Strasbourg', "In

In [231]:
# Checking if any null values exist
print(list_2024_wta.isnull().sum())

# Total number of missing values
print("Total missing values:", list_2024_wta.isnull().sum().sum())

tournament_name    0
city               0
country            0
winner             3
dtype: int64
Total missing values: 3


As soon as our tournament lists have the same columns, we can proceed to concatenating them and making 1 general list with unique values

In [234]:
# Concatenating four tables into one
tournaments_list = pd.concat([list_2023_atp, list_2023_wta, list_2024_atp, list_2024_wta], ignore_index=True)

# Renaming column name
tournaments_list.rename(columns={'tournament_name': 'Tournament name'}, inplace=True)

# Making 1 list with unique values only
tournaments_list.drop_duplicates()
tournaments_list

,Tournament name,city,country,winner
0,Adelaide International,Adelaide,Australia,Novak Djokovic
1,Tata Open Maharashtra,Pune,India,Tallon Griekspoor
2,ASB Classic,Auckland,New Zealand,Richard Gasquet
3,Adelaide International 2,Adelaide,Australia,SoonWoo Kwon
4,Australian Open,Melbourne,Australia,Novak Djokovic
...,...,...,...,...
332,Mundo Tenis Open,Florianópolis,Brazil,Maja Chwalinska
333,Open BLS de Limoges,Limoges,France,Viktorija Golubic
334,Brisbane International Presented by Evie,Brisbane,Australia,Aryna Sabalenka
335,ASB Classic,Auckland,New Zealand,Clara Tauson


### Checking unique tags

In [238]:
# Flattening the lists in the 'tags' column and get unique values
unique_tags = set(tag for tags in video_df['tags'].dropna() for tag in tags)

# Converting to a list
unique_tags_list = list(unique_tags)

print(unique_tags_list)

['swiatek vs cirstea', 'Monfils 2018', 'dep potro 2017', 'hanfmann mpetschi perricard', 'L. Samsonova vs. B. Krejcikova', 'Mario Balotelli', 'tsonga del potro', 'jannik sinner rome 2023', 'ATP Handshakes', 'Greet Minnen vs. Liudmila Samsonova', 'Danielle Collins vs. Elena Rybakina', 'dimitrov alcaraz', 'carlos alcaraz paris', 'berettini', 'parks vs ruse', 'djokovic vs musetti miami open 2025', 'Demon', 'alcaraz medevdev', 'Edmund Highlights', 'Schwartzman 2022', 'US Open 2024', '2023 Citi Open', 'tennis youtube shorts', 'S. Stephens vs. E. Raducanu', 'kalinskaya', 'wuhan open', 'Draper Lehecka', 'grass courts', 'Shelton drop shot', 'fils vs rune highlights', 'ben shelton arthur fils', 'Eusebio 2007', 'Roger Federer 2024', 'Korea Open 2024', 'top shots', 'monfils speed', 'indian wells best moments', 'D. Yastremska vs. A. Li', 'alcaraz v tsitsipas', 'Kokkinakis hilarious', 'Struff 2024', 'Shapovalov', 'Ben Shelton vs Jannik Sinner Highlights', 'Medvedev Milan', 'Funny sport', 'alcaraz wi

In [240]:
# Flattening the list of tags and count occurrences
tag_counts = Counter(tag for tags in video_df['tags'].dropna() for tag in tags)

# Converting to a DataFrame for better readability (optional)
tag_counts_df = pd.DataFrame(tag_counts.items(), columns=['Tag', 'Count'])

# Printing the tag counts
print(tag_counts_df)

                                       Tag  Count
0                                   Tennis   4527
1                                    Sport   3854
2                                   Sports   3852
3                                      ATP   3868
4                                 ATP Tour   3872
...                                    ...    ...
16650  Sloane Stephens vs. Aryna Sabalenka      1
16651                       San Diego Open      1
16652               C. Garcia vs. N. Osaka      1
16653       D. Kasatkina vs. Y. Putintseva      1
16654            I. Swiatek vs. M. Sakkari      1

[16655 rows x 2 columns]


### Let's try to distinguish the tournament name by finding similarities in the city name

In [243]:
# Step 1: Getting unique, non-null city names
cities = tournaments_list['city'].dropna().unique()

# Step 2: Defining a function to find matching cities in a video title
def find_matching_cities(title, cities):
    if not isinstance(title, str):
        return []
    return [city for city in cities if city.lower() in title.lower()]

# Step 3: Applying the function to each video title and joining the results
video_df['Matched_Cities'] = video_df['title'].apply(lambda t: ', '.join(find_matching_cities(t, cities)) or 'No Match')

# Step 4: Previewing the result
print(video_df[['video_id', 'title', 'Matched_Cities']].head())

       video_id                                              title  \
92  6QZRf6UhyOY                         Holger Rune Pool Dive 🏊‍♂️   
93  fZh5q73PX6A  Carlos Alcaraz vs Holger Rune For The Title! 🏆...   
94  lsxIGuoVLMw           Holger Rune Lifts The Barcelona Title! 🏆   
95  l2kxJP23H8M  Holger Rune Defeats Alcaraz To Win The Title I...   
96  NwDHg7xpZoo             FC Barcelona Players In The Building 🤩   

   Matched_Cities  
92       No Match  
93      Barcelona  
94      Barcelona  
95      Barcelona  
96      Barcelona  


In [245]:
# Calculating No Match values
no_match_count = (video_df['Matched_Cities'] == 'No Match').sum()
print(f"Number of video titles with no matching city: {no_match_count}")
total_videos = len(video_df)
percentage_no_match = (no_match_count / total_videos) * 100
print(f"That's {percentage_no_match:.2f}% of all videos.")


Number of video titles with no matching city: 2605
That's 37.37% of all videos.


### Let's try to distinguish the tournament name by finding similarities in the playlist name

In [248]:
# Step 1: Defining a function to find matching cities in a video title
tournament_names = tournaments_list['Tournament name'].dropna().unique()

def find_matching_in_playlist_name(playlist_name, tournament_names):
    if not isinstance(playlist_name, str):
        return []
    return [tournament_name for tournament_name in tournament_names if tournament_name.lower() in playlist_name.lower()]

# Step 2: Applying the function to each video title and joining the results
video_df['Matched_Playlist_Titles'] = video_df['playlist_title'].apply(lambda t: ', '.join(find_matching_in_playlist_name(t, tournament_names)) or 'No Match')

# Step 3: Previewing the result
print(video_df[['video_id', 'title', 'Matched_Playlist_Titles']].head())

       video_id                                              title  \
92  6QZRf6UhyOY                         Holger Rune Pool Dive 🏊‍♂️   
93  fZh5q73PX6A  Carlos Alcaraz vs Holger Rune For The Title! 🏆...   
94  lsxIGuoVLMw           Holger Rune Lifts The Barcelona Title! 🏆   
95  l2kxJP23H8M  Holger Rune Defeats Alcaraz To Win The Title I...   
96  NwDHg7xpZoo             FC Barcelona Players In The Building 🤩   

   Matched_Playlist_Titles  
92                No Match  
93                No Match  
94                No Match  
95                No Match  
96                No Match  


In [250]:
# Calculating No Match values
no_match_count2 = (video_df['Matched_Playlist_Titles'] == 'No Match').sum()
print(f"Number of video titles with no playlist title: {no_match_count2}")
total_videos = len(video_df)
percentage_no_match2 = (no_match_count2 / total_videos) * 100
print(f"That's {percentage_no_match2:.2f}% of all videos.")

Number of video titles with no playlist title: 5254
That's 75.37% of all videos.


In [252]:
# Checking unique values of Matched Playlst Titles
print(video_df['Matched_Playlist_Titles'].unique())

['No Match' 'Nitto ATP Finals' 'Australian Open' 'Mutua Madrid Open'
 'Rolex Monte-Carlo Masters' 'BNP Paribas Open'
 'Adelaide International, Adelaide International 2'
 'Adelaide International, Adelaide International 1' 'Rolex Paris Masters'
 'Porsche Tennis Grand Prix' 'ATX Open'
 'Dubai Duty Free Tennis Championships' 'Transylvania Open'
 'Upper Austria Ladies Linz' 'Hobart International'
 'Adelaide International' 'ASB Classic' 'Mérida Open Akron' 'Jiangxi Open'
 'WTA Finals' 'Guangzhou Open' 'Kinoshita Group Japan Open' 'Ningbo Open'
 'China Open' 'Thailand Open' 'Korea Open'
 'Guadalajara Open Akron presented by Santander' 'Abierto GNP Seguros'
 'Tennis in the Land powered by Rocket Mortgage' 'Cincinnati Open'
 'Mubadala Citi DC Open' 'Unicredit Iasi Open' 'Livesport Prague Open'
 'Palermo Ladies Open' 'Hungarian Grand Prix' 'Libéma Open'
 'Internationaux de Strasbourg' "Internazionali BNL d'Italia"
 'Open Capfinances Rouen Métropole' 'Credit One Charleston Open'
 'Cymbiotika San 

### Let's try to distinguish the tournament name by finding similarities in the playlist name and city names

In [255]:
# Step 1: Getting unique, non-null city names
cities = tournaments_list['city'].dropna().unique()

# Step 2: Defining a function to find matching cities in a video title
def find_matching_cities_in_playlist(playlist_title, cities):
    if not isinstance(playlist_title, str):
        return []
    return [city for city in cities if city.lower() in playlist_title.lower()]

# Step 3: Applying the function to each video title and join results
video_df['Matched_Cities_and_Playlist_titles'] = video_df['playlist_title'].apply(lambda t: ', '.join(find_matching_cities_in_playlist(t, cities)) or 'No Match')

# Step 4: Previewing result
print(video_df[['video_id', 'title', 'Matched_Cities_and_Playlist_titles']].head())

       video_id                                              title  \
92  6QZRf6UhyOY                         Holger Rune Pool Dive 🏊‍♂️   
93  fZh5q73PX6A  Carlos Alcaraz vs Holger Rune For The Title! 🏆...   
94  lsxIGuoVLMw           Holger Rune Lifts The Barcelona Title! 🏆   
95  l2kxJP23H8M  Holger Rune Defeats Alcaraz To Win The Title I...   
96  NwDHg7xpZoo             FC Barcelona Players In The Building 🤩   

   Matched_Cities_and_Playlist_titles  
92                          Barcelona  
93                          Barcelona  
94                          Barcelona  
95                          Barcelona  
96                          Barcelona  


In [257]:
# Calculating No Match values

no_match_count3 = (video_df['Matched_Cities_and_Playlist_titles'] == 'No Match').sum()
print(f"Number of video titles with no playlist title: {no_match_count3}")
total_videos = len(video_df)
percentage_no_match3 = (no_match_count3 / total_videos) * 100
print(f"That's {percentage_no_match3:.2f}% of all videos.")

Number of video titles with no playlist title: 2721
That's 39.03% of all videos.


In [265]:
no_match_df = video_df[video_df['Matched_Cities_and_Playlist_titles'] == 'No Match'].copy()

In [267]:
len(no_match_df)

1629

In [271]:
# Exploring No Match
no_match_df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,commentCount,duration,definition,caption,playlist_id,playlist_title,durationSeconds,tagCount,Matched_Cities,Matched_Playlist_Titles,Matched_Cities_and_Playlist_titles
229,ezuE1Yez7J8,Tennis TV,"Alcaraz vs Hurkacz, Dimitrov vs Monfils in Bes...",SUBSCRIBE to our channel for the best ATP tenn...,"[Tennis, Sport, Sports, ATP, ATP Tour, Hot Sho...",2025-04-03 15:00:52+00:00,174249.0,939.0,43.0,PT33M23S,hd,false,PLQHHr8gPOsH59fyLiE3QdQHTz5E1wRhOH,Best ATP Tennis Matches Of 2025,2003.0,26,No Match,No Match,No Match
245,pBCY4J_K6hU,Tennis TV,Rune's Epic Tweener; Moutet Behind-The-Back & ...,SUBSCRIBE to our channel for the best ATP tenn...,"[Tennis, Sport, Sports, ATP, ATP Tour, Hot Sho...",2025-04-01 12:30:57+00:00,26281.0,548.0,27.0,PT14M5S,hd,false,PLQHHr8gPOsH5uGeYg-TN8wHRy-9j8yPPM,Best Shots & Rallies: 2025 ATP Tennis Season,845.0,39,No Match,No Match,No Match
246,VcX2fZ8Cfe8,Tennis TV,"Alcaraz Behind The Back, Bellucci Tweener & Pa...",SUBSCRIBE to our channel for the best ATP tenn...,"[Tennis, Sport, Sports, ATP, ATP Tour, Hot Sho...",2025-03-01 10:47:38+00:00,64264.0,907.0,48.0,PT12M13S,hd,false,PLQHHr8gPOsH5uGeYg-TN8wHRy-9j8yPPM,Best Shots & Rallies: 2025 ATP Tennis Season,733.0,39,No Match,No Match,No Match
247,eSWrjnuDTwM,Tennis TV,The Best Shots From The First Week of Tennis i...,"Looking at the best shots from Hong Kong, Bris...","[tennis, tennis tv, sports highlights, tennis ...",2025-01-06 15:15:23+00:00,28908.0,416.0,17.0,PT9M55S,hd,false,PLQHHr8gPOsH5uGeYg-TN8wHRy-9j8yPPM,Best Shots & Rallies: 2025 ATP Tennis Season,595.0,18,No Match,No Match,No Match
248,x7HLxjxiHAg,Tennis TV,"Crazy Celebrations, On-Court Pepsi & Tennis In...",SUBSCRIBE to our channel for the best ATP tenn...,"[Tennis, Sport, Sports, ATP, ATP Tour, Hot Sho...",2025-04-05 15:30:30+00:00,30127.0,418.0,26.0,PT10M,hd,false,PLQHHr8gPOsH6qncixKFsQ9zQ50aHz2XZY,Weird Tennis 🤪,600.0,30,No Match,No Match,No Match
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13093,h-s-knWcHAA,WTA,Describe the Olympic Sport 😎,Can you guess the five different Olympic sport...,"[tennis highlights, tennis, tenis, WTA, Women'...",2024-07-28 18:32:36+00:00,7624.0,198.0,22.0,PT4M58S,hd,false,PLhQBpwasxUpnc4uO-1LH6Vh70pGTvabM6,WTA Exclusives,298.0,11,No Match,No Match,No Match
13094,YQ1OHYbIshI,WTA,"Jenga with a Twist 😜 Hunter, Perez, Parks, Ste...","Pick a block, answer the question and repeat!\...","[tennis highlights, tennis, tenis, WTA, Women'...",2023-12-01 16:00:17+00:00,7544.0,135.0,5.0,PT3M52S,hd,false,PLhQBpwasxUpnc4uO-1LH6Vh70pGTvabM6,WTA Exclusives,232.0,11,No Match,No Match,No Match
13095,HOPESXIzX3Y,WTA,WTA players share their go-to Chipotle order 🌯,"Next in line, what can I get started for you 🌯...","[tennis highlights, tennis, tenis, WTA, Women'...",2023-08-21 15:39:23+00:00,14371.0,206.0,24.0,PT2M22S,hd,false,PLhQBpwasxUpnc4uO-1LH6Vh70pGTvabM6,WTA Exclusives,142.0,11,No Match,No Match,No Match
13096,apWJM0bhZtY,WTA,WTA takes on the biggest night in Hollywood! 🎬,How many tennis balls stack up to the height o...,"[tennis highlights, tennis, tenis, WTA, Women'...",2023-03-12 17:35:24+00:00,36746.0,808.0,59.0,PT3M39S,hd,false,PLhQBpwasxUpnc4uO-1LH6Vh70pGTvabM6,WTA Exclusives,219.0,11,No Match,No Match,No Match


In [273]:
# Creating a mask
mask = (video_df['Matched_Cities_and_Playlist_titles'] == 'No Match') & (video_df['Matched_Cities'] != 'No Match')

video_df.loc[mask, 'Matched_Cities_and_Playlist_titles'] = video_df.loc[mask, 'Matched_Cities']

In [275]:
# Calculating No Match values
no_match_count3 = (video_df['Matched_Cities_and_Playlist_titles'] == 'No Match').sum()
print(f"Number of video titles with no playlist title: {no_match_count3}")
total_videos = len(video_df)
percentage_no_match3 = (no_match_count3 / total_videos) * 100
print(f"That's {percentage_no_match3:.2f}% of all videos.")

Number of video titles with no playlist title: 1629
That's 23.37% of all videos.


In [279]:
# Updating and taking a look in no_match_df again
no_match_df = video_df[video_df['Matched_Cities_and_Playlist_titles'] == 'No Match'].copy()

In [281]:
# Let's see where we can troubleshoot and retrieve more information of tournament
no_match_df['playlist_title'].unique()

array(['Best ATP Tennis Matches Of 2025',
       'Best Shots & Rallies: 2025 ATP Tennis Season', 'Weird Tennis 🤪',
       'Rio Open 2025: February 17-23', 'Tiebreaks with a Twist 🤪',
       'Novak Djokovic vs Andy Murray: Epic Rivalry!',
       '2025 ATP Tennis Season: Final Highlights',
       'United Cup 2025: Dec 27 - Jan 5',
       '2024 ATP Player Highlight Reels 🎬',
       'Best Of The 2024 ATP Tennis Season!',
       'Practice At Nitto ATP Finals 2024',
       'Nitto ATP Finals 2024: Nov 10-17', 'Six Kings Slam: Oct 16-19',
       'Montreal 2024: August 6-12', "Queen's 2024: 17th - 23rd June",
       'Libema Open 2024: June 10 - June 17',
       'Best Clay-Court Matches Of 2024 🧱',
       'Best ATP Tennis Matches Of 2024 Season!',
       'The Best of Jannik Sinner',
       'Australian Open: ATP Tour Moments To Watch',
       'Most Improbable ATP Comebacks!',
       'Funniest Moments Of The 2024 Season 🤣',
       'Rio 2024: February 19th - 25th',
       '2024 ATP Tennis Season: F

In [283]:
print(tournaments_list['Tournament name'].tolist())

['Adelaide International', 'Tata Open Maharashtra', 'ASB Classic', 'Adelaide International 2', 'Australian Open', 'Cordoba Open', 'Open Occitanie', 'Dallas Open', 'ABN Amro Open', 'Delray Beach Open', 'IEB+ Argentina Open', 'Dubai Duty Free Tennis Championships', 'Qatar Exxon Mobil Open', 'Rio Open presented by Claro', 'Open 13 Provence', 'Movistar Chile Open', 'Abierto Mexicano Telcel presentado por HSBC', 'BNP Paribas Open', 'Miami Open presented by Itau', "Fayez Sarofim & Co. U.S. Men's Clay Court Championship", 'Millennium Estoril Open', 'Grand Prix Hassan II', 'Rolex Monte-Carlo Masters', 'BMW Open by Bitpanda', 'Barcelona Open Banc Sabadell', 'Srpska Open Banja', 'Mutua Madrid Open', "Internazionali BNL d'Italia", 'Gonet Geneva Open', 'Open Parc Auvergne-Rhone-Alpes Lyon', 'Roland Garros', 'Boss Open', 'Libéma Open', 'Terra Wortmann Open', 'HSBC Championships', 'Lexus Eastbourne Open', 'Mallorca Championships', 'Wimbledon', 'Nordea Open', 'EFG Swiss Open Gstaad', 'Infosys Hall of

Since our def function missed some values there is clearly room for the improvement

In [306]:
tqdm.pandas()

def improved_tournament_match(video_df, tournaments_list, threshold=85):
    # Getting unique cities and tournaments
    tournament_names = tournaments_list['Tournament name'].dropna().unique()
    cities = tournaments_list['city'].dropna().unique()

    # Normalising text: remove punctuation, lowercase
    def normalize(text):
        return re.sub(r'[^\w\s]', '', text.lower()) if isinstance(text, str) else ''

    # Fuzzy match helper
    def best_fuzzy_match(text, candidates):
        best_match = 'No Match'
        best_score = 0
        for cand in candidates:
            score = fuzz.partial_ratio(normalize(text), normalize(cand))
            if score > best_score:
                best_match = cand
                best_score = score
        return best_match if best_score >= threshold else 'No Match'

    # Matching tournament from playlist title or video title
    def match_row(row):
        playlist_title = row.get('playlist_title', '')
        video_title = row.get('title', '')

        match_from_playlist = best_fuzzy_match(playlist_title, tournament_names)
        if match_from_playlist != 'No Match':
            return match_from_playlist
        
        # If no match from playlist, try video title
        match_from_title = best_fuzzy_match(video_title, tournament_names)
        if match_from_title != 'No Match':
            return match_from_title

        # Fallback: match by city
        for city in cities:
            if city.lower() in normalize(playlist_title) or city.lower() in normalize(video_title):
                tournaments_in_city = tournaments_list[tournaments_list['city'].str.lower() == city.lower()]
                if not tournaments_in_city.empty:
                    return tournaments_in_city['Tournament name'].values[0]
        return 'No Match'

    # Applying matching logic
    video_df['Matched_Tournament_Name'] = video_df.progress_apply(match_row, axis=1)
    return video_df

In [308]:
# Applying a function
video_df = improved_tournament_match(video_df, tournaments_list)

100%|██████████| 6971/6971 [00:29<00:00, 238.13it/s]


In [310]:
# Exploring gow many were not matched
no_match_df = video_df[video_df['Matched_Tournament_Name'] == 'No Match']
print(no_match_df['playlist_title'].value_counts().head(10))
print(f"Unmatched count: {len(no_match_df)} out of {len(video_df)}")

playlist_title
Short Tennis Videos                                 414
Monte-Carlo 2024: April 7th - 14th                   75
Monte-Carlo 2025: 6 - 13 April                       69
Montreal 2024: August 6-12                           50
ATP Tennis Compilations                              48
#Shorts                                              39
2023 Omnium Banque Nationale présenté par Rogers     29
NETFLIX: BREAK POINT 📺                               29
Best Of The 2024 ATP Tennis Season!                  29
United Cup 2023: 29 December - 8 January             28
Name: count, dtype: int64
Unmatched count: 1423 out of 6971


In [312]:
# Checking Matched_Tournament_Name with No Match values and playlist_title counts of those values
# Filtering videos with no matched tournament name
no_match_df = video_df[video_df['Matched_Tournament_Name'] == 'No Match']

# Counting occurrences of each playlist title among those
playlist_counts = no_match_df['playlist_title'].value_counts()

# Displaying the result
print(playlist_counts)

playlist_title
Short Tennis Videos                             414
Monte-Carlo 2024: April 7th - 14th               75
Monte-Carlo 2025: 6 - 13 April                   69
Montreal 2024: August 6-12                       50
ATP Tennis Compilations                          48
                                               ... 
Cologne 2 2020: October 19-25                     1
2024 National Bank Open                           1
Best ATP Tennis Matches Of 2025                   1
When Passion Meets Performance | Rolex x WTA      1
Off-Court Activities                              1
Name: count, Length: 101, dtype: int64


In [314]:
# Unifying Shorts playlists 
# Defining replacements
playlist_replacements = {
    '#Shorts': 'Short Tennis Videos'
}

# Performing replacements
video_df.loc[:, 'playlist_title'] = video_df['playlist_title'].replace(playlist_replacements)

# Dobule-Checking
print("Unique playlist titles after replacement:")
video_df['playlist_title'].value_counts()

Unique playlist titles after replacement:


playlist_title
Short Tennis Videos                             534
Miami 2025: March 19-30                         121
Madrid 2024: April 24th - May 5th               118
Nitto ATP Finals 2024: Nov 10-17                118
Indian Wells 2024: March 6th - 17th             114
                                               ... 
Antwerp 2022: October 17-23                       1
When Passion Meets Performance | Rolex x WTA      1
The Best of Matteo Berrettini                     1
Nitto ATP Finals Turin 2021: November 14-21       1
Off-Court Activities                              1
Name: count, Length: 383, dtype: int64

In [316]:
# Filtering videos with no matched tournament name
no_match_df = video_df[video_df['Matched_Tournament_Name'] == 'No Match']

# Grouping by playlist title and getting counts and viewCount sum
playlist_stats = no_match_df.groupby('playlist_title').agg(
    video_count=('video_id', 'count'),
    total_views=('viewCount', 'sum')
).sort_values(by='video_count', ascending=False)

# Displaying the result
print(playlist_stats)


                                          video_count  total_views
playlist_title                                                    
Short Tennis Videos                               453  106212748.0
Monte-Carlo 2024: April 7th - 14th                 75   20296341.0
Monte-Carlo 2025: 6 - 13 April                     69   11825163.0
Montreal 2024: August 6-12                         50    6528188.0
ATP Tennis Compilations                            48    9582986.0
...                                               ...          ...
Cologne 2 2020: October 19-25                       1     471343.0
10 MINUTES OF                                       1     340046.0
Best ATP Tennis Matches Of 2025                     1     174249.0
Queen's 2022: June 13-19                            1       9593.0
Funniest Tennis Moments in 2022 Season 😂            1      30364.0

[100 rows x 2 columns]


In [320]:
# Defining a function to display video_df stats
def summarize_video_stats(video_df):
    # 1. Total stats
    total_videos = len(video_df)
    total_views = video_df['viewCount'].sum()

    # 2. Unmatched tournament stats
    no_match_df = video_df[video_df['Matched_Tournament_Name'] == 'No Match']
    unmatched_videos = len(no_match_df)
    unmatched_views = no_match_df['viewCount'].sum()

    # 3. "Short Tennis Videos" stats
    shorts_df = video_df[video_df['playlist_title'] == 'Short Tennis Videos']
    shorts_videos = len(shorts_df)
    shorts_views = shorts_df['viewCount'].sum()

    # 4. Percentages
    unmatched_video_pct = (unmatched_videos / total_videos) * 100
    unmatched_view_pct = (unmatched_views / total_views) * 100

    shorts_video_pct = (shorts_videos / total_videos) * 100
    shorts_view_pct = (shorts_views / total_views) * 100

    # 5. Displaying
    print(f"🎾 Total Videos: {total_videos:,}")
    print(f"👁️  Total Views: {int(total_views):,}\n")

    print(f"🚫 Unmatched Tournament Name:")
    print(f"   Videos: {unmatched_videos:,} ({unmatched_video_pct:.2f}%)")
    print(f"   Views:  {int(unmatched_views):,} ({unmatched_view_pct:.2f}%)\n")

    print(f"🎬 'Short Tennis Videos' Playlist:")
    print(f"   Videos: {shorts_videos:,} ({shorts_video_pct:.2f}%)")
    print(f"   Views:  {int(shorts_views):,} ({shorts_view_pct:.2f}%)")


In [322]:
summarize_video_stats(video_df)

🎾 Total Videos: 6,971
👁️  Total Views: 1,093,257,110

🚫 Unmatched Tournament Name:
   Videos: 1,423 (20.41%)
   Views:  273,317,478 (25.00%)

🎬 'Short Tennis Videos' Playlist:
   Videos: 534 (7.66%)
   Views:  131,676,327 (12.04%)


As soon as Unmatched Tournament Name consists almost half of all the videos we should handle No Match values as much as possible

In [325]:
# Let's see how many rows we can troubleshoot by hands
print(sorted(no_match_df['playlist_title'].unique().tolist()))

["'s-Hertogenbosch 2023: June 12-18", '10 MINUTES OF', '2019 Next Gen ATP Finals, Milan: November 5-9', '2023 ATP Tennis Season: Final Highlights', '2023 Copa Colsanitas', '2023 Omnium Banque Nationale présenté par Rogers', '2023 Player Highlight Reels', '2023 Rothesay Open', '2023 United Cup', '2024 ATP Player Highlight Reels 🎬', '2024 ATP Tennis Season: Final Highlights', '2024 National Bank Open', '2024 Wrap', '2025 ATP Tennis Season: Final Highlights', 'ARCHIVE GOLD: Top 20 ATP Shots & Rallies From Years Gone By', 'ATP Cup 2022', 'ATP Tennis Compilations', 'ATP Tennis Funny Moments & Fails', 'Ajla Tomljanovic | AS SEEN ON NETFLIX: BREAK POINT 🎥', 'BEST ATP PLAYER RIVALRIES', 'Best ATP Tennis Matches In 2023!', 'Best ATP Tennis Matches Of 2024 Season!', 'Best ATP Tennis Matches Of 2025', 'Best ATP Tennis Shots & Rallies in 2023 Season!', 'Best Clay-Court Matches Of 2024 🧱', 'Best Of Nadal vs Djokovic!', 'Best Of The 2022 ATP Tennis Season!', 'Best Of The 2024 ATP Tennis Season!', 'B

In [327]:
# Manually puttin tournament names  

# Creating override
playlist_to_tournament_override = {
    's-Hertogenbosch 2023': 'Libéma Open',
    '2019 Coupe Rogers, Montreal: August 5-11': 'National Bank Open presented by Rogers',
    '2019 Next Gen ATP Finals, Milan: November 5-9': 'Next Gen ATP Finals presented by PIF',
    '2023 Copa Colsanitas': 'Copa Colsanitas Zurich presentado por VISA',
    '2023 Omnium Banque Nationale présenté par Rogers': 'National Bank Open presented by Rogers',
    '2023 Rothesay Open': 'Lexus Nottingham Open',
    '2023 United Cup': 'United Cup',
    '2024 National Bank Open': 'National Bank Open presented by Rogers',
    'ATP Cup 2022': 'ATP Cup',
    'Cologne 2 2020: October 19-25': 'Cologne 2',
    'Monte-Carlo 2024: April 7th - 14th': 'Rolex Monte-Carlo Masters', 
    'Monte-Carlo 2025: 6 - 13 April': 'Rolex Monte-Carlo Masters',
    'Monte-Carlo Classic Moments': 'Rolex Monte-Carlo Masters',
    'Montreal 2022: August 7-14': 'National Bank Open presented by Rogers', 
    'Montreal 2024: August 6-12': 'National Bank Open presented by Rogers',
    "Queen's 2022: June 13-19": 'HSBC Championships', # different quotation marks for avoiding errors
    "Queen's 2023: June 19-25": 'HSBC Championships', # different quotation marks for avoiding errors
    "Queen's 2024: 17th - 23rd June": 'HSBC Championships',# different quotation marks for avoiding errors
    'Rio 2024: February 19th - 25th': 'Rio Open presented by Claro', 
    'Rio Open 2020: February 17-23': 'Rio Open presented by Claro', 
    'Rio Open 2023: February 20-26': 'Rio Open presented by Claro', 
    'Rio Open 2025: February 17-23': 'Rio Open presented by Claro',
    'Six Kings Slam: Oct 16-19': 'Six Kings Slam',
    'United Cup 2023: 29 December - 8 January': 'United Cup', 
    'United Cup 2024: December 29 - January 7': 'United Cup', 
    'United Cup 2025: Dec 27 - Jan 5': 'United Cup',
    'Winston-Salem 2023: August 20-26': 'Winston-Salem Open',    
    'Next Gen ATP Finals 2017, Milan | November 7-11': 'Next Gen ATP Finals presented by PIF',
    'Next Gen ATP Finals 2023': 'Next Gen ATP Finals presented by PIF',
    'Tie Break Tens Doubles In The Desert: March 7th': 'BNP Paribas Open'
}

In [329]:
# Defining a function to imply overriding
def apply_manual_tournament_fixes(video_df, override_map):
    for keyword, tournament_name in override_map.items():
        mask = (
            (video_df['Matched_Tournament_Name'] == 'No Match') &
            (video_df['playlist_title'].str.contains(keyword, case=False, na=False))
        )
        video_df.loc[mask, 'Matched_Tournament_Name'] = tournament_name
    return video_df


In [331]:
# Applying a ovverride function
video_df = apply_manual_tournament_fixes(video_df, playlist_to_tournament_override)

In [333]:
# Showing rows where the tournament name is now in override_map (meaning they were updated)
overridden_titles = list(playlist_to_tournament_override.values())

# Filtering only those rows where the value was changed from 'No Match' to a known override
updated_df = video_df[video_df['Matched_Tournament_Name'].isin(overridden_titles)]

# Displaying the results
print("\n🎯 Rows affected by manual override:")
display(updated_df[['video_id', 'title', 'playlist_title', 'Matched_Tournament_Name']])



🎯 Rows affected by manual override:


,video_id,title,playlist_title,Matched_Tournament_Name
151,yCBrz4zYWEM,Tennis Player Impressions: The Players Themsel...,Monte-Carlo 2025: 6 - 13 April,Rolex Monte-Carlo Masters
152,xlvoBzlswtE,Alcaraz On Nadal’s Clay Legacy 🧱🗣️,Monte-Carlo 2025: 6 - 13 April,Rolex Monte-Carlo Masters
153,YVXaItU2wbM,Carlos Alcaraz's Best Shots From Title Winning...,Monte-Carlo 2025: 6 - 13 April,Rolex Monte-Carlo Masters
154,E4gkmUVcXAA,Lorenzo Musetti & Carlos Alcaraz React To 2025...,Monte-Carlo 2025: 6 - 13 April,Rolex Monte-Carlo Masters
155,Zr0hccethQw,"Alcaraz Tweeners, 48-Shot Rallies & Spin Back ...",Monte-Carlo 2025: 6 - 13 April,Rolex Monte-Carlo Masters
...,...,...,...,...
12437,X1lh9-1QR1g,Story of the Tournament | 2023 United Cup,2023 United Cup,United Cup
12438,fnl9Mf8eSz0,All the BEST off-court moments 😜 | 2023 United...,2023 United Cup,United Cup
13745,w9wUjBXBisw,FULL MATCH | Elena Rybakina vs. Aryna Sabalenk...,Full Matches & Condensed Match Highlights,BNP Paribas Open
13852,HZnFhZCEakQ,CONDENSED MATCH | Jasmine Paolini vs. Aryna Sa...,Full Matches & Condensed Match Highlights,BNP Paribas Open


In [335]:
# Checking the impact
summarize_video_stats(video_df)

🎾 Total Videos: 6,971
👁️  Total Views: 1,093,257,110

🚫 Unmatched Tournament Name:
   Videos: 961 (13.79%)
   Views:  197,714,065 (18.08%)

🎬 'Short Tennis Videos' Playlist:
   Videos: 534 (7.66%)
   Views:  131,676,327 (12.04%)


In [337]:
# Getting column names
print(video_df.columns)

Index(['video_id', 'channelTitle', 'title', 'description', 'tags',
       'publishedAt', 'viewCount', 'likeCount', 'commentCount', 'duration',
       'definition', 'caption', 'playlist_id', 'playlist_title',
       'durationSeconds', 'tagCount', 'Matched_Cities',
       'Matched_Playlist_Titles', 'Matched_Cities_and_Playlist_titles',
       'Matched_Tournament_Name'],
      dtype='object')


In [339]:
# Dropping unnecessary columns
video_df.drop(columns=['Matched_Playlist_Titles', 'Matched_Cities_and_Playlist_titles'], inplace=True)

# Renaming columns
video_df.rename(columns={'Matched_Cities': 'City', 'Matched_Tournament_Name': 'Tournament name'}, inplace=True)

# Verifying the results
print(video_df.columns)

Index(['video_id', 'channelTitle', 'title', 'description', 'tags',
       'publishedAt', 'viewCount', 'likeCount', 'commentCount', 'duration',
       'definition', 'caption', 'playlist_id', 'playlist_title',
       'durationSeconds', 'tagCount', 'City', 'Tournament name'],
      dtype='object')


# Creating a table with players' names and matching with videos

#### Let's obtain a list of all tennis players, both male and  female, as soon as we are going to analyse both ATP and WTA 

In [343]:
# Setting URLs of both male and female tennis players names for scraping
male_url = 'https://en.wikipedia.org/wiki/List_of_male_singles_tennis_players'
female_url = 'https://en.wikipedia.org/wiki/List_of_female_tennis_players'

In [345]:
# Defining a function to extract players information
def extract_players(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    players = []

    tables = soup.find_all('table', {'class': 'wikitable'})

    for table in tqdm(tables, desc=f"Processing tables from {url}"):
        rows = table.find_all('tr')[1:]  # Skip header
        for row in tqdm(rows, leave=False, desc="Parsing rows"):
            cols = row.find_all('td')
            if len(cols) >= 2:
                # Get player name
                name = cols[0].get_text(strip=True)

                # Try to extract country by looking for flag icon (more reliable)
                country = None
                for col in cols[1:]:
                    img = col.find('img')
                    if img and 'flagicon' in img.get('class', []):
                        country = col.get_text(strip=True)
                        break
                    elif img:  # fallback if 'flagicon' class not set
                        country = col.get_text(strip=True)
                        break

                # If no flag, fallback to second column as best guess
                if not country and len(cols) > 1:
                    country = cols[1].get_text(strip=True)

                players.append({'name': name, 'country': country})
    return players

In [347]:
# Getting players
male_players = extract_players(male_url)
female_players = extract_players(female_url)

Processing tables from https://en.wikipedia.org/wiki/List_of_male_singles_tennis_players:   0%|          | 0/1 [00:00<?, ?it/s]
Parsing rows:   0%|          | 0/679 [00:00<?, ?it/s]
Processing tables from https://en.wikipedia.org/wiki/List_of_male_singles_tennis_players: 100%|██████████| 1/1 [00:00<00:00, 44.02it/s]
Processing tables from https://en.wikipedia.org/wiki/List_of_female_tennis_players:   0%|          | 0/1 [00:00<?, ?it/s]
Parsing rows:   0%|          | 0/575 [00:00<?, ?it/s]
Processing tables from https://en.wikipedia.org/wiki/List_of_female_tennis_players: 100%|██████████| 1/1 [00:00<00:00, 64.94it/s]


NB! Since the lists contains some restrictions, we need to put some names in manually. For the purpose of further analysis, we will do this only for Ukrainian and Romanian female tennis players who still have active careers as players

In [350]:
female_players.extend([
    {'name': 'Elina Svitolina', 'country': 'Ukraine'},
    {'name': 'Marta Kostyuk', 'country': 'Ukraine'},
    {'name': 'Dayana Yastremska', 'country': 'Ukraine'},
    {'name': 'Anhelina Kalinina', 'country': 'Ukraine'},
    {'name': 'Yuliia Starodubtseva', 'country': 'Ukraine'},
    {'name': 'Anastasiia Sobolieva', 'country': 'Ukraine'},
    {'name': 'Lesia Tsurenko', 'country': 'Ukraine'},
    {'name': 'Valeriya Strakhova', 'country': 'Ukraine'},
    {'name': 'Oleksandra Oliynykova', 'country': 'Ukraine'},
    {'name': 'Katarina Zavatska', 'country': 'Ukraine'},
    {'name': 'Veronika Podrez', 'country': 'Ukraine'},
    {'name': 'Kateryna Baindl', 'country': 'Ukraine'},
    {'name': 'Anastasiya Lopata', 'country': 'Ukraine'},
    {'name': 'Kateryna Volodko', 'country': 'Ukraine'},
    {'name': 'Yelizaveta Kotliar', 'country': 'Ukraine'},
    {'name': 'Nadiia Kolb', 'country': 'Ukraine'},
    {'name': 'Daria Yesypchuk', 'country': 'Ukraine'},
    {'name': 'Kateryna Lazarenko', 'country': 'Ukraine'},
    {'name': 'Alisa Baranovska', 'country': 'Ukraine'},
    {'name': 'Anita Sahdiieva', 'country': 'Ukraine'},
    {'name': 'Anastasiya Zaparyniuk', 'country': 'Ukraine'},
    {'name': 'Anastasiia Firman', 'country': 'Ukraine'},
    {'name': 'Mariya Poduraeva', 'country': 'Ukraine'},
    {'name': 'Anastasiya Poplavska', 'country': 'Ukraine'},
    {'name': 'Mariia Bergen', 'country': 'Ukraine'},
    {'name': 'Krystyna Pochtovyk', 'country': 'Ukraine'},
    {'name': 'Yelyzaveta Chainykova', 'country': 'Ukraine'},
    {'name': 'Elena Grekul', 'country': 'Ukraine'},
    {'name': 'Khrystyna Vozniak', 'country': 'Ukraine'},    
    {'name': 'Lyudmyla Kichenok', 'country': 'Ukraine'},
    {'name': 'Nadiia Kichenok', 'country': 'Ukraine'},
    {'name': 'Maryna Kolb', 'country': 'Ukraine'},
    {'name': 'Mariya Kostyuk', 'country': 'Ukraine'},
    {'name': 'Aliesia Reva', 'country': 'Ukraine'},

# Romanian female tennis players
    {'name': 'Jaqueline Cristian', 'country': 'Romania'},
    {'name': 'Irina-Camelia Begu', 'country': 'Romania'},
    {'name': 'Elena-Gabriela Ruse', 'country': 'Romania'},
    {'name': 'Anca Todoni', 'country': 'Romania'},
    {'name': 'Sorana Cirstea', 'country': 'Romania'},
    {'name': 'Ana Bogdan', 'country': 'Romania'},
    {'name': 'Miriam Bulgaru', 'country': 'Romania'},
    {'name': 'Patricia-Maria Tig', 'country': 'Romania'},
    {'name': 'Irina Bara', 'country': 'Romania'},
    {'name': 'Georgia Craciun', 'country': 'Romania'},
    {'name': 'Cristina Dinu', 'country': 'Romania'},
    {'name': 'Gabriela Lee', 'country': 'Romania'},
    {'name': 'Ilinca Amariei', 'country': 'Romania'},
    {'name': 'Andreea Prisacariu', 'country': 'Romania'},
    {'name': 'Lavinia Tanaise', 'country': 'Romania'},
    {'name': 'Oana-Georgeta Simion', 'country': 'Romania'},
    {'name': 'Stefania Bojica', 'country': 'Romania'},
    {'name': 'Andreea Mitu', 'country': 'Romania'},
    {'name': 'Oana Gavrila', 'country': 'Romania'},
    {'name': 'Elena-Ruxandra Bertea', 'country': 'Romania'},
    {'name': 'Bianca Barbulescu', 'country': 'Romania'},
    {'name': 'Irina Fetecau', 'country': 'Romania'},
    {'name': 'Maria-Sara Popa', 'country': 'Romania'},
    {'name': 'Karola Bejenaru', 'country': 'Romania'},
    {'name': 'Arina Vasilescu', 'country': 'Romania'},
    {'name': 'Ioana Zvonaru', 'country': 'Romania'},
    {'name': 'Carmen-Andreea Herea', 'country': 'Romania'},
    {'name': 'Eva-Maria Ionescu', 'country': 'Romania'},
    {'name': 'Briana Szabo', 'country': 'Romania'},
    {'name': 'Alexia-Iulia Marginean', 'country': 'Romania'},
    {'name': 'Elena-Teodora Cadar', 'country': 'Romania'},
    {'name': 'Maria Toma', 'country': 'Romania'},
    {'name': 'Alexandra Iordache', 'country': 'Romania'},
    {'name': 'Anamaria Oana', 'country': 'Romania'},
    {'name': 'Anastasia Safta', 'country': 'Romania'},
    {'name': 'Mara Gae', 'country': 'Romania'},
    {'name': 'Simona Ogescu', 'country': 'Romania'},
    {'name': 'Patricia Goina', 'country': 'Romania'},
    {'name': 'Giulia-Safina Popa', 'country': 'Romania'},
    {'name': 'Vanessa Popa-Teiusanu', 'country': 'Romania'},
    {'name': 'Ioana-Teodora Sava', 'country': 'Romania'},
    {'name': 'Diana-Ioana Simionescu', 'country': 'Romania'},
    {'name': 'Alexia-Lavinia Puiac', 'country': 'Romania'},
    {'name': 'Andreea Rosca', 'country': 'Romania'},
    {'name': 'Evelyne Tiron', 'country': 'Romania'},
    {'name': 'Stefana Lazar', 'country': 'Romania'},
    {'name': 'Monica Niculescu', 'country': 'Romania'},
    {'name': 'Alexandra-Irina Anghel', 'country': 'Romania'},
    {'name': 'Cara-Maria Mester', 'country': 'Romania'},
    {'name': 'Cristiana Todoni', 'country': 'Romania'},
    {'name': 'Diana Mihail', 'country': 'Romania'},
    {'name': 'Ilinca Sagmar', 'country': 'Romania'},
    {'name': 'Iulia-Andreea Ionescu', 'country': 'Romania'},
    {'name': 'Alexia-Shara Iancu', 'country': 'Romania'},
    {'name': 'Maia-Ilinca Burcescu', 'country': 'Romania'}
])
# and fix Iga, since during Data Visualization in Tableau Iga was not shown
for player in female_players:
    if player['name'] == 'Iga Świątek':
        player['name'] = 'Iga Swiatek'

In [352]:
# Combining results
all_players = male_players + female_players

# Converting to DataFrame
players_df = pd.DataFrame(all_players)

# Previewing
print(players_df.head())

                 name        country
0   Robert Abdesselam         France
1        José Acasuso      Argentina
2  András Ádám-Stolpa        Hungary
3        Andre Agassi  United States
4       Ronald Agénor          Haiti


In [354]:
# Checking unique countries
unique_countries = players_df['country'].dropna().unique().tolist()

# Displaying
print(f"Total unique countries: {len(unique_countries)}")
print("List of countries:")
print(sorted(unique_countries))

# Checking unique players
unique_players = players_df['name'].dropna().unique().tolist()

# Displaying the number of tennis players
print(f"\nTotal unique players: {len(unique_players)}")

Total unique countries: 104
List of countries:
['/Czechoslovakia/Egypt', '/Germany', '/South Africa', '/Spain', 'Argentina', 'ArgentinaPeru', 'Australia', 'Australia/South Africa', 'AustraliaSlovakia', 'AustraliaUnited Kingdom', 'AustraliaYugoslaviaSerbia andMontenegro', 'Austria', 'Belarus', 'Belgium', 'Bermuda', 'Bohemia', 'Brazil', 'Bulgaria', 'Canada', 'Chile', 'China', 'Chinese Taipei', 'Colombia', 'Croatia', 'Cyprus', 'Czech Republic', 'Czechia', 'Czechoslovakia', 'Czechoslovakia /Czech Republic', 'Czechoslovakia/United States', 'CzechoslovakiaAustralia', 'CzechoslovakiaCzech Republic', 'CzechoslovakiaHungary', 'CzechoslovakiaUnited States', 'Denmark', 'Ecuador', 'Egypt', 'Estonia', 'Finland', 'France', 'Georgia', 'Germany', 'GermanyDenmark', 'Great Britain', 'Greece', 'Haiti', 'Hungary', 'India', 'Indonesia', 'Ireland', 'Israel', 'Italy', 'Japan', 'Kazakhstan', 'Latvia', 'Luxembourg', 'Mexico', 'Morocco', 'Netherlands', 'New Zealand', 'Norway', 'NorwayUnited States', 'Paraguay',

In [356]:
# Let's take a quick glance if names were parced right
print(unique_players)

['Robert Abdesselam', 'José Acasuso', 'András Ádám-Stolpa', 'Andre Agassi', 'Ronald Agénor', 'Juan Aguilera', 'Karim Alami', 'Carlos Alcaraz', 'Fred Alexander', 'John Alexander', 'Wilmer Allison', 'Nicolás Almagro', 'Manuel Alonso', 'Felicisimo Ampon', 'Vijay Amritraj', 'Mario Ančić', 'James Anderson', 'Kevin Anderson', 'Mal Anderson', 'Igor Andreev', 'John Andrews', 'Matt Anger', 'Paul Annacone', 'Hicham Arazi', 'Jimmy Arias', 'Jordi Arrese', 'József Asbóth', 'Arthur Ashe', 'Félix Auger-Aliassime', 'Bunny Austin', 'Luis Ayala', 'Wilfred Baddeley', 'Sebastián Báez', 'Marcos Baghdatis', 'Corrado Barazzutti', 'Herbert Roper Barrett', 'Pierre Barthès', 'Nikoloz Basilashvili', 'Roberto Bautista Agut', 'Alfred Beamish', 'Boris Becker', 'Karl Behr', 'Mike Belkin', 'Alphonzo Bell', 'Thomaz Bellucci', 'Eduardo Bengoechea', 'Julien Benneteau', 'Alberto Berasategui', 'Tomáš Berdych', 'Lennart Bergelin', 'Jay Berger', 'Christian Bergström', 'Marcel Bernard', 'Matteo Berrettini', 'Paolo Bertolucci

In [358]:
# Normalising and changing to the last known country
# Defining replacements
country_replacements = {
    '/Czechoslovakia/Egypt': 'Egypt',
    '/Germany': 'Germany',
    '/South Africa': 'South Africa',
    '/Spain': 'Spain',
    'ArgentinaPeru': 'Peru',
    'Australia/South Africa': 'South Africa',
    'AustraliaSlovakia': 'Slovakia',
    'AustraliaUnited Kingdom': 'United Kingdom',
    'AustraliaYugoslaviaSerbia andMontenegro': 'Serbia and Montenegro',
    'Czechoslovakia /Czech Republic': 'Czech Republic',
    'Czechoslovakia/United States': 'United States',
    'CzechoslovakiaAustralia': 'Australia',
    'CzechoslovakiaCzech Republic': 'Czech Republic',
    'CzechoslovakiaHungary': 'Hungary',
    'CzechoslovakiaUnited States': 'United States',
    'GermanyDenmark': 'Denmark',
    'NorwayUnited States': 'United States',
    'Peru /United States': 'United States',
    'Puerto RicoUnited States': 'United States',
    'RussiaKazakhstan': 'Kazakhstan',
    'South Africa /United States': 'United States',
    'South Africa/United States': 'United States',
    'South AfricaUnited States': 'United States',
    'Soviet Union /Ukraine': 'Ukraine',
    'Soviet Union/Russia': 'Russia',
    'Soviet UnionGeorgia': 'Georgia',
    'Soviet UnionRussia': 'Russia',
    'Soviet UnionUkraine': 'Ukraine',
    'UzbekistanUnited States': 'United States',
    'West Germany/': 'Germany',
    'West Germany/Germany': 'Germany',
    'Yugoslavia /Croatia': 'Croatia',
    'YugoslaviaUnited States': 'United States',
}

# Performing replacements
players_df.loc[:, 'country'] = players_df['country'].replace(country_replacements)

# Double-Checking
print("Unique countries of players after replacement:")
print(sorted(players_df['country'].unique().tolist()))

Unique countries of players after replacement:
['Argentina', 'Australia', 'Austria', 'Belarus', 'Belgium', 'Bermuda', 'Bohemia', 'Brazil', 'Bulgaria', 'Canada', 'Chile', 'China', 'Chinese Taipei', 'Colombia', 'Croatia', 'Cyprus', 'Czech Republic', 'Czechia', 'Czechoslovakia', 'Denmark', 'Ecuador', 'Egypt', 'Estonia', 'Finland', 'France', 'Georgia', 'Germany', 'Great Britain', 'Greece', 'Haiti', 'Hungary', 'India', 'Indonesia', 'Ireland', 'Israel', 'Italy', 'Japan', 'Kazakhstan', 'Latvia', 'Luxembourg', 'Mexico', 'Morocco', 'Netherlands', 'New Zealand', 'Norway', 'Paraguay', 'Peru', 'Philippines', 'Poland', 'Puerto Rico', 'Rhodesia', 'Romania', 'Russia', 'Serbia', 'Serbia and Montenegro', 'Slovakia', 'Slovenia', 'South Africa', 'Soviet Union', 'Spain', 'Sweden', 'Switzerland', 'Thailand', 'Tunisia', 'Ukraine', 'United Kingdom', 'United States', 'Uruguay', 'Uzbekistan', 'West Germany', 'Yugoslavia', 'Zimbabwe']


In [362]:
# Checking null values
null_name_count = players_df['name'].isnull().sum()
null_country_count = players_df['country'].isnull().sum()
print(f"Number of null values in 'name' column: {null_name_count}")
print(f"Number of null values in 'country' column: {null_country_count}")

Number of null values in 'name' column: 0
Number of null values in 'country' column: 0


In [364]:
# Saving players table
players_df.to_excel('players_df.xlsx', index=False)

Preparing for identyfing players appereances in videos

In [377]:
# Preparing player lists
players_df = players_df.dropna(subset=['name'])
players_df['first_name'] = players_df['name'].apply(lambda x: x.split()[0])
players_df['last_name'] = players_df['name'].apply(lambda x: x.split()[-1].lower())

# Maps and sets
full_names = set(players_df['name'])
last_name_map = {row['last_name']: row['name'] for _, row in players_df.iterrows()}
ambiguous_last_names = {k for k, v in players_df['last_name'].value_counts().items() if v > 1}

# Defining ambiguous standalone words (not strictly needed anymore)
ambiguous_last_names = {'court', 'king', 'love', 'white', 'black', 'brown', 'young'}

# Building player patterns and last name uniqueness map
def build_player_patterns(players_df):
    patterns = []
    last_name_counts = {}

    for _, row in players_df.iterrows():
        full_name = row['name']
        name_parts = full_name.split()
        if len(name_parts) >= 2:
            first_name, last_name = name_parts[0], name_parts[-1].lower()
            patterns.append({
                'full_name': full_name,
                'first_name': first_name.lower(),
                'last_name': last_name
            })
            last_name_counts[last_name] = last_name_counts.get(last_name, 0) + 1

    return patterns, last_name_counts

player_patterns, last_name_counts = build_player_patterns(players_df)

def extract_players(text):
    if not isinstance(text, str):
        return []
    text = text.lower()

    candidates = set()
    
    tokens = re.split(r'[,\|\-:;]| vs\.?| v\.?', text)
    tokens = [t.strip() for t in tokens if t.strip()]
    
    for token in tokens:
        words = set(re.findall(r'\b\w+\b', token))
        for player in player_patterns:
            full_name_lower = player['full_name'].lower()
            last_name = player['last_name']

            if full_name_lower in token:
                candidates.add(player['full_name'])
            elif (
                last_name in words 
                and last_name_counts[last_name] == 1  # Only match unique last names
                and last_name not in ambiguous_last_names
            ):
                candidates.add(player['full_name'])

    return sorted(candidates)

def define_players(video_df):
    video_df['players_from_title'] = video_df['title'].progress_apply(extract_players)

    video_df['players'] = video_df.apply(
        lambda row: row['players_from_title'] if row['players_from_title'] else extract_players(row['playlist_title']),
        axis=1
    )

    video_df['players'] = video_df['players'].apply(lambda x: ', '.join(x) if x else 'No Match')

    video_df.drop(columns=['players_from_title'], inplace=True)

    return video_df


In [381]:
# Applying a function
video_df = define_players(video_df)
print(video_df[['title', 'playlist_title', 'players']].head())


100%|██████████| 6971/6971 [00:03<00:00, 1807.66it/s]


                                                title  \
92                         Holger Rune Pool Dive 🏊‍♂️   
93  Carlos Alcaraz vs Holger Rune For The Title! 🏆...   
94           Holger Rune Lifts The Barcelona Title! 🏆   
95  Holger Rune Defeats Alcaraz To Win The Title I...   
96             FC Barcelona Players In The Building 🤩   

                   playlist_title                      players  
92  Barcelona 2025: 14 - 20 April                  Holger Rune  
93  Barcelona 2025: 14 - 20 April  Carlos Alcaraz, Holger Rune  
94  Barcelona 2025: 14 - 20 April                  Holger Rune  
95  Barcelona 2025: 14 - 20 April  Carlos Alcaraz, Holger Rune  
96  Barcelona 2025: 14 - 20 April                     No Match  


In [383]:
# Exploring the results
video_df.head()

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,commentCount,duration,definition,caption,playlist_id,playlist_title,durationSeconds,tagCount,City,Tournament name,players
92,6QZRf6UhyOY,Tennis TV,Holger Rune Pool Dive 🏊‍♂️,SUBSCRIBE to our channel for the best ATP tenn...,"[Tennis, Sport, Sports, ATP, ATP Tour, Hot Sho...",2025-04-20 17:20:27+00:00,237861.0,5237.0,108.0,PT52S,hd,false,PLQHHr8gPOsH6KEvZs8n8-l4UXJuLz9GJn,Barcelona 2025: 14 - 20 April,52.0,27,No Match,Barcelona Open Banc Sabadell,Holger Rune
93,fZh5q73PX6A,Tennis TV,Carlos Alcaraz vs Holger Rune For The Title! 🏆...,SUBSCRIBE to our channel for the best ATP tenn...,"[Tennis, Sport, Sports, ATP, ATP Tour, Hot Sho...",2025-04-20 16:58:29+00:00,782691.0,11565.0,1104.0,PT8M1S,hd,false,PLQHHr8gPOsH6KEvZs8n8-l4UXJuLz9GJn,Barcelona 2025: 14 - 20 April,481.0,21,Barcelona,Barcelona Open Banc Sabadell,"Carlos Alcaraz, Holger Rune"
94,lsxIGuoVLMw,Tennis TV,Holger Rune Lifts The Barcelona Title! 🏆,SUBSCRIBE to our channel for the best ATP tenn...,"[Tennis, Sport, Sports, ATP, ATP Tour, Hot Sho...",2025-04-20 16:39:11+00:00,44144.0,1411.0,56.0,PT27S,hd,false,PLQHHr8gPOsH6KEvZs8n8-l4UXJuLz9GJn,Barcelona 2025: 14 - 20 April,27.0,24,Barcelona,Barcelona Open Banc Sabadell,Holger Rune
95,l2kxJP23H8M,Tennis TV,Holger Rune Defeats Alcaraz To Win The Title I...,SUBSCRIBE to our channel for the best ATP tenn...,"[Tennis, Sport, Sports, ATP, ATP Tour, Hot Sho...",2025-04-20 16:08:37+00:00,69790.0,2351.0,101.0,PT1M11S,hd,false,PLQHHr8gPOsH6KEvZs8n8-l4UXJuLz9GJn,Barcelona 2025: 14 - 20 April,71.0,24,Barcelona,Barcelona Open Banc Sabadell,"Carlos Alcaraz, Holger Rune"
96,NwDHg7xpZoo,Tennis TV,FC Barcelona Players In The Building 🤩,SUBSCRIBE to our channel for the best ATP tenn...,"[Tennis, Sport, Sports, ATP, ATP Tour, Hot Sho...",2025-04-20 14:46:25+00:00,74229.0,1037.0,8.0,PT14S,hd,false,PLQHHr8gPOsH6KEvZs8n8-l4UXJuLz9GJn,Barcelona 2025: 14 - 20 April,14.0,27,Barcelona,Barcelona Open Banc Sabadell,No Match


### To perform future analysis let's devide playlist names into categories

In [385]:
# Checking unique playlist_titles to define categories and keywords for categorising
playlist_titles_list = video_df['playlist_title'].unique().tolist()
print(sorted(playlist_titles_list))

["'s-Hertogenbosch 2023: June 12-18", '10 MINUTES OF', '2019 Next Gen ATP Finals, Milan: November 5-9', '2023 ASB Classic', '2023 ATP Tennis Season: Final Highlights', '2023 ATX Open', '2023 Abierto GNP Seguros', '2023 Adelaide International 1', '2023 Adelaide International 1: January 1-8', '2023 Adelaide International 2', '2023 Adelaide International 2: January 9-14', '2023 BNP Paribas Open', '2023 BNP Paribas Warsaw Open', '2023 Bad Homburg Open presented by Engel & Volkers', '2023 China Open', '2023 Copa Colsanitas', '2023 Credit One Charleston Open', '2023 Cymbiotika San Diego Open', '2023 Dubai Duty Free Tennis Championships', '2023 GNP Seguros WTA Finals Cancun', '2023 Galaxy Holding Group Guangzhou Open', '2023 Grand Prix Son Altesse Royale La Princesse Lalla Meryem', '2023 Guadalajara Open AKRON', '2023 Hamburg European Open', '2023 Hana Bank Korea Open', '2023 Hobart International', '2023 Internationaux de Strasbourg', "2023 Internazionali BNL d'Italia", '2023 Jasmin Open Mona

In [387]:
# Defining a function for categorising playlists
def categorise_playlist(row):
    title = str(row['playlist_title']).lower()
    
    # 1. Tournaments
    if row['Tournament name'] != 'No Match':
        return 'Tournaments'
    
    # 2. Highlights
    highlights_keywords = [
        'best of', 'highlight', 'extended highlights', 'player highlight',
        'most memorable', 'memorable match', 'comeback', 'final highlights',
        'season highlights', 'retired player highlights', '#ThankYouSerena', 
        '#Fedal', '10 MINUTES OF', 'Hot Shots', '2024 Wrap', 
        'ARCHIVE GOLD: Top 20 ATP Shots & Rallies From Years Gone By', 
        'ATP God Mode Moments!', 'Compilations', 'ATP World Tour Classic Moments and Matches',
        'BEST ATP PLAYER RIVALRIES', 'Best Tennis Matches', 'Best Shots', 'Best Tennis Shots', 'Best Tennis Rallies',
        'Best Clay-Court', 'Best Court-Level Tennis Highlights!', 'Best Shots & Rallies', 'Classic Moments', 
        'Full Matches & Condensed Match Highlights', 'Generation Game: When Generations Collide!', 'LEGENDS OF TENNIS', 
        'Marcelo Rios Magic', 'Most Improbable', 'Best Moments', 'Epic Rivalry', 'Our Must-Watch Tennis Videos Right Now',
        'Practice At', 'Tiebreaks with a Twist', 'Underrated'
        
    ]
    if any(kw in title for kw in highlights_keywords):
        return 'Highlights'

    # 3. Shorts
    if 'short tennis videos' in title:
        return 'Shorts'

    # 4. Netflix
    if 'netflix' in title or 'break point' in title:
        return 'Netflix'

    # 5. Exclusives & Others
    exclusive_keywords = [
        'the real me', 'modern health', 'what moves you', 'off-court',
        "what's goin ons", 'morgan stanley', 'wta x', 'x wta', 'ATP Access All Areas: Challenges, Games & More',
        "FINALS DAY: Behind The Scenes on Great Days in Tennis Players' Lives", #different quotation marks for avoiding error
        'Inside The Tour', 'LOVE-15', 'WTA Exclusives', 'Behind The Scenes'
    ]
    if any(kw in title for kw in exclusive_keywords):
        return 'Exclusives & Others'

    # 6. Dramatic & Funny
    funny_keywords = ['funny', 'funniest', 'fails', 'dramatic', 'weird']
    if any(kw in title for kw in funny_keywords):
        return 'Dramatic & Funny'

    # Default fallback
    return 'Other'

In [391]:
video_df = video_df.copy()
video_df['playlist_category'] = video_df.progress_apply(categorise_playlist, axis=1)

100%|██████████| 6971/6971 [00:00<00:00, 145017.82it/s]


In [395]:
category_counts = video_df['playlist_category'].value_counts()
print(category_counts)

playlist_category
Tournaments            6010
Shorts                  453
Highlights              222
Other                   172
Dramatic & Funny         47
Netflix                  42
Exclusives & Others      25
Name: count, dtype: int64


In [405]:
# Filtering df to troubleshoot 'Other' playlist category
other_playlist_df = video_df[video_df['playlist_category'] == 'Other']

# Explore the results
print(sorted(other_playlist_df['playlist_title'].unique()))

['10 MINUTES OF', '2024 Wrap', 'ARCHIVE GOLD: Top 20 ATP Shots & Rallies From Years Gone By', 'ATP Tennis Compilations', 'BEST ATP PLAYER RIVALRIES', 'Best ATP Tennis Matches In 2023!', 'Best ATP Tennis Matches Of 2024 Season!', 'Best ATP Tennis Matches Of 2025', 'Best ATP Tennis Shots & Rallies in 2023 Season!', 'Best Clay-Court Matches Of 2024 🧱', 'Best Shots & Rallies: 2025 ATP Tennis Season', 'Best Shots Of The 2023 Tennis Season! 🍿', 'Carlos Alcaraz vs Jannik Sinner: The Rivalry 🤝', 'Carlos Alcaraz vs Stefanos Tsitsipas: The Rivalry', "FINALS DAY: Behind The Scenes on Great Days in Tennis Players' Lives", 'Inside The Tour', 'LOVE-15 🎾', 'Marcelo Rios Magic', 'Novak Djokovic vs Andy Murray: Epic Rivalry!', 'Our Must-Watch Tennis Videos Right Now 👀', 'Roger Federer Best Shots at Each Masters 1000!', 'Tiebreaks with a Twist 🤪', 'WTA Exclusives']


In [407]:
# Defining updates
category_updates = {
    '10 MINUTES OF': 'Highlights',
    '2024 Wrap': 'Highlights',
    'ARCHIVE GOLD: Top 20 ATP Shots & Rallies From Years Gone By': 'Highlights',
    'ATP Tennis Compilations': 'Highlights',
    'BEST ATP PLAYER RIVALRIES': 'Highlights',
    'Best ATP Tennis Matches In 2023!': 'Highlights',
    'Best ATP Tennis Matches Of 2024 Season!': 'Highlights',
    'Best ATP Tennis Matches Of 2025': 'Highlights',
    'Best ATP Tennis Shots & Rallies in 2023 Season!': 'Highlights',
    'Best Clay-Court Matches Of 2024 🧱': 'Highlights',
    'Best Shots & Rallies: 2025 ATP Tennis Season': 'Highlights',
    'Best Shots Of The 2023 Tennis Season! 🍿': 'Highlights',
    'Carlos Alcaraz vs Jannik Sinner: The Rivalry 🤝': 'Highlights',
    'Carlos Alcaraz vs Stefanos Tsitsipas: The Rivalry': 'Highlights',
    "FINALS DAY: Behind The Scenes on Great Days in Tennis Players' Lives": 'Exclusives & Others',
    'Inside The Tour': 'Exclusives & Others',
    'LOVE-15 🎾': 'Exclusives & Others',
    'Marcelo Rios Magic': 'Highlights',
    'Novak Djokovic vs Andy Murray: Epic Rivalry!': 'Highlights',
    'Our Must-Watch Tennis Videos Right Now 👀': 'Highlights',
    'Roger Federer Best Shots at Each Masters 1000!': 'Highlights',
    'Tiebreaks with a Twist 🤪': 'Highlights',
    'WTA Exclusives': 'Exclusives & Others'
}

# Applying the updates
video_df['playlist_category'] = video_df.apply(
    lambda row: category_updates[row['playlist_title']] 
    if row['playlist_title'] in category_updates 
    else row['playlist_category'],
    axis=1
)

# Double-Checking
category_counts = video_df['playlist_category'].value_counts()
print(category_counts)

playlist_category
Tournaments            5945
Shorts                  453
Highlights              431
Exclusives & Others      53
Dramatic & Funny         47
Netflix                  42
Name: count, dtype: int64


# Creating a dataframe with players' appearances in videos

Creating the 1st DF with summary for each player

In [412]:
# Initialising defaultdicts for counting
player_data = defaultdict(lambda: {
    "video_count": 0,
    "total_views": 0,
    "total_likes": 0,
    "total_comments": 0,
    "video_ids": set()
})

# Iterating through rows and split player names
for _, row in video_df.iterrows():
    players = [p.strip() for p in row['players'].split(',') if p.strip() and p != 'No Match']
    video_id = row['video_id']
    views = row.get('viewCount', 0)
    likes = row.get('likeCount', 0)
    comments = row.get('commentCount', 0)
    
    for player in players:
        player_data[player]["video_count"] += 1
        player_data[player]["total_views"] += views
        player_data[player]["total_likes"] += likes
        player_data[player]["total_comments"] += comments
        player_data[player]["video_ids"].add(video_id)

# Converting to DataFrame
summary_df = pd.DataFrame([
    {
        "player_name": player,
        "video_count": data["video_count"],
        "total_views": data["total_views"],
        "total_likes": data["total_likes"],
        "total_comments": data["total_comments"],
        "video_ids": list(data["video_ids"])
    }
    for player, data in player_data.items()
])

# Saving
summary_df.to_excel("player_video_summary.xlsx", index=False)

In [414]:
summary_df

,player_name,video_count,total_views,total_likes,total_comments,video_ids
0,Holger Rune,215,48806803.0,687003.0,57587.0,"[bP2e2XAfSI0, pubq0Ddhgaw, -uccdUfpyAI, bDUgAm..."
1,Carlos Alcaraz,661,198053344.0,3465473.0,187686.0,"[jUrm2jyxnYw, N2hdp_xLhrI, I2m2ltWi8FA, Fgy8tj..."
2,Arthur Fils,98,15922189.0,231283.0,22668.0,"[-5IUEqcD6Xw, 07ktqSKJk8o, -7a5vlDGSDQ, pBwpzd..."
3,Karen Khachanov,76,13626738.0,174403.0,15416.0,"[XujgbA94fx8, c9u5FROS4NU, rSgfshr5HGQ, qr1Oi7..."
4,Alex de Minaur,135,26642847.0,394214.0,33100.0,"[rNAHKcCEUeQ, 5__PQ-Smy2Y, KB_qUeAzsLo, UVCioe..."
...,...,...,...,...,...,...
196,Kristina Mladenovic,1,35895.0,384.0,32.0,[m6Fro_W7z3M]
197,Garbiñe Muguruza,3,294467.0,3082.0,559.0,"[bc6947J0l6E, qp23cCcnltU, 69c4Ky2HBVM]"
198,Fred Perry,1,192799.0,2608.0,73.0,[O4DHGEr0060]
199,Betty Rosenquest Pratt,1,4463.0,91.0,9.0,[YtcK6hogT5U]


Creating the 2nd DF with information with separate video_ids for better setting relations in Tableau

In [417]:
# Creating a flat player–video mapping DataFrame
flat_player_video_rows = []

# Iterating
for _, row in video_df.iterrows():
    video_id = row['video_id']
    views = row['viewCount']
    likes = row.get('likeCount', 0)
    comments = row.get('commentCount', 0)
    players = [p.strip() for p in row['players'].split(',') if p.strip() and p != 'No Match']
    
    for player in players:
        flat_player_video_rows.append({
            "player_name": player,
            "video_id": video_id,
            "views": views,
            "likes": likes,
            "comments": comments
        })

# Converting to DataFrame
player_video_df = pd.DataFrame(flat_player_video_rows)

# Saving
player_video_df.to_excel("player_video_df.xlsx", index=False)


In [419]:
player_video_df

,player_name,video_id,views,likes,comments
0,Holger Rune,6QZRf6UhyOY,237861.0,5237.0,108.0
1,Carlos Alcaraz,fZh5q73PX6A,782691.0,11565.0,1104.0
2,Holger Rune,fZh5q73PX6A,782691.0,11565.0,1104.0
3,Holger Rune,lsxIGuoVLMw,44144.0,1411.0,56.0
4,Carlos Alcaraz,l2kxJP23H8M,69790.0,2351.0,101.0
...,...,...,...,...,...
9418,Aryna Sabalenka,RBzmO1sUpTw,11444.0,107.0,3.0
9419,Madison Keys,RBzmO1sUpTw,11444.0,107.0,3.0
9420,Iga Swiatek,iTt5bh5b6WQ,10141.0,91.0,9.0
9421,Maria Sakkari,iTt5bh5b6WQ,10141.0,91.0,9.0


# Final Data Cleaning

### Cleaning channels

In [423]:
# Veryfing channels
video_df['channelTitle'].value_counts()

channelTitle
Tennis TV                    4802
WTA                          2164
Rolex Monte-Carlo Masters       3
Rolex Shanghai Masters          2
Name: count, dtype: int64

In [425]:
# Defining replacements
channels_replacements = {
    'Rolex Monte-Carlo Masters': 'Tennis TV',
    'Rolex Shanghai Masters': 'Tennis TV'
}

# Applying replacements
video_df.loc[:, 'channelTitle'] = video_df['channelTitle'].replace(channels_replacements)

# Double-Checking
print("Unique channel titles after replacement:")
video_df['channelTitle'].value_counts()

Unique channel titles after replacement:


channelTitle
Tennis TV    4807
WTA          2164
Name: count, dtype: int64

### Dividing players by columns

In [427]:
# Let's see how many maximum players we have in one video for further deviding by columns
longest_players_value = video_df['players'].loc[video_df['players'].str.len().idxmax()]
print(longest_players_value)


Casper Ruud, Geoff Masters, Grigor Dimitrov, Holger Rune, Stefanos Tsitsipas, Taylor Fritz, Tommy Paul


In [429]:
# Defining a function to split players for columns
def split_players_to_columns(df, source_col='players', max_players=8):
    # Split by comma and expand
    players_split = df[source_col].str.split(',', n=max_players-1, expand=True)
    
    # Assign to new player columns, if they exist
    for i in range(players_split.shape[1]):
        df[f'player_{i+1}'] = players_split[i].str.strip()

    return df


In [431]:
# Applying function
video_df = split_players_to_columns(video_df)

In [433]:
# Exploring the results
video_df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,commentCount,duration,...,Tournament name,players,playlist_category,player_1,player_2,player_3,player_4,player_5,player_6,player_7
92,6QZRf6UhyOY,Tennis TV,Holger Rune Pool Dive 🏊‍♂️,SUBSCRIBE to our channel for the best ATP tenn...,"[Tennis, Sport, Sports, ATP, ATP Tour, Hot Sho...",2025-04-20 17:20:27+00:00,237861.0,5237.0,108.0,PT52S,...,Barcelona Open Banc Sabadell,Holger Rune,Tournaments,Holger Rune,None,None,None,None,None,None
93,fZh5q73PX6A,Tennis TV,Carlos Alcaraz vs Holger Rune For The Title! 🏆...,SUBSCRIBE to our channel for the best ATP tenn...,"[Tennis, Sport, Sports, ATP, ATP Tour, Hot Sho...",2025-04-20 16:58:29+00:00,782691.0,11565.0,1104.0,PT8M1S,...,Barcelona Open Banc Sabadell,"Carlos Alcaraz, Holger Rune",Tournaments,Carlos Alcaraz,Holger Rune,None,None,None,None,None
94,lsxIGuoVLMw,Tennis TV,Holger Rune Lifts The Barcelona Title! 🏆,SUBSCRIBE to our channel for the best ATP tenn...,"[Tennis, Sport, Sports, ATP, ATP Tour, Hot Sho...",2025-04-20 16:39:11+00:00,44144.0,1411.0,56.0,PT27S,...,Barcelona Open Banc Sabadell,Holger Rune,Tournaments,Holger Rune,None,None,None,None,None,None
95,l2kxJP23H8M,Tennis TV,Holger Rune Defeats Alcaraz To Win The Title I...,SUBSCRIBE to our channel for the best ATP tenn...,"[Tennis, Sport, Sports, ATP, ATP Tour, Hot Sho...",2025-04-20 16:08:37+00:00,69790.0,2351.0,101.0,PT1M11S,...,Barcelona Open Banc Sabadell,"Carlos Alcaraz, Holger Rune",Tournaments,Carlos Alcaraz,Holger Rune,None,None,None,None,None
96,NwDHg7xpZoo,Tennis TV,FC Barcelona Players In The Building 🤩,SUBSCRIBE to our channel for the best ATP tenn...,"[Tennis, Sport, Sports, ATP, ATP Tour, Hot Sho...",2025-04-20 14:46:25+00:00,74229.0,1037.0,8.0,PT14S,...,Barcelona Open Banc Sabadell,No Match,Tournaments,No Match,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13860,y4yqpksTQk8,WTA,CONDENSED MATCH | Daria Kasatkina vs. Yulia Pu...,Watch the condensed highlights of the Hologic ...,"[D. Kasatkina, D. Kasatkina vs. Y. Putintseva,...",2025-01-18 02:13:37+00:00,5004.0,119.0,4.0,PT20M12S,...,Ningbo Open,Daria Kasatkina,Tournaments,Daria Kasatkina,None,None,None,None,None,None
13861,RBzmO1sUpTw,WTA,When Sabalenka & Keys Met In Beijing! | WTA Co...,Watch the condensed highlights of the Hologic ...,"[A. Sabalenka, A. Sabalenka vs. M. Keys, Aryna...",2025-01-24 16:24:00+00:00,11444.0,107.0,3.0,PT20M6S,...,China Open,"Aryna Sabalenka, Madison Keys",Tournaments,Aryna Sabalenka,Madison Keys,None,None,None,None,None
13862,iTt5bh5b6WQ,WTA,Iga Swiatek vs. Maria Sakkari | Indian Wells 2...,Watch the condensed highlights of the Hologic ...,"[I. Swiatek, I. Swiatek vs. M. Sakkari, Iga Sw...",2025-03-03 17:00:46+00:00,10141.0,91.0,9.0,PT20M7S,...,BNP Paribas Open,"Iga Swiatek, Maria Sakkari",Tournaments,Iga Swiatek,Maria Sakkari,None,None,None,None,None
15981,plfcEN6bV28,WTA,Aryna Sabalenka in Wuhan: Top Points from 2018...,Watch Aryna Sabalenka's best plays as she went...,"[tennis highlights, tennis, tenis, WTA, Women'...",2024-10-08 19:00:27+00:00,6659.0,89.0,7.0,PT12M54S,...,Dongfeng Voyah Wuhan Open,Aryna Sabalenka,Tournaments,Aryna Sabalenka,None,None,None,None,None,None


### Change column names and drop temporary columns that were used for matching

In [436]:
# Checking column names
print(video_df.columns.tolist())

['video_id', 'channelTitle', 'title', 'description', 'tags', 'publishedAt', 'viewCount', 'likeCount', 'commentCount', 'duration', 'definition', 'caption', 'playlist_id', 'playlist_title', 'durationSeconds', 'tagCount', 'City', 'Tournament name', 'players', 'playlist_category', 'player_1', 'player_2', 'player_3', 'player_4', 'player_5', 'player_6', 'player_7']


In [438]:
# Renaming columns
video_df = video_df.rename(columns={
    'channelTitle': 'channel_title',
    'title': 'video_title',
    'description': 'video_description',
    'publishedAt': 'publication_date',
    'viewCount': 'views',
    'likeCount': 'likes',
    'commentCount': 'comments',
    'tagCount': 'tag_count',
    'City': 'Related City',
    'players': 'players_names'
})

In [440]:
# Excluding timezone information
video_df['publication_date'] = video_df['publication_date'].dt.tz_localize(None)

In [442]:
# Defining a function to remove emojis
def remove_emojis(text):
    if not isinstance(text, str):
        return text
    emoji_pattern = re.compile(
        "[" 
        "\U0001F600-\U0001F64F"  # Emoticons
        "\U0001F300-\U0001F5FF"  # Symbols & pictographs
        "\U0001F680-\U0001F6FF"  # Transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # Flags
        "\U00002700-\U000027BF"  # Dingbats
        "\U0001F900-\U0001F9FF"  # Supplemental symbols
        "\U00002600-\U000026FF"  # Misc symbols
        "\U00002B00-\U00002BFF"  # Misc symbols and arrows
        "\U0001FA70-\U0001FAFF"  # Symbols and pictographs extended-A
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# Applying a function
video_df['video_title'] = video_df['video_title'].apply(remove_emojis)

In [444]:
# During Data Vizualisation in Tableau, I faced the issue with 'When Paire & Tsonga Switched Sports ️' title of 'uZQiKjXypg0' video_id	
# Removing trailing whitespace and non-printing characters (like emojis)
video_df['video_title'] = video_df['video_title'].apply(lambda x: re.sub(r'[\s\W_]+$', '', x))

In [446]:
# Saving
video_df.to_excel('video_df.xlsx', index=False)

# Date of pulling info from API: 1 May 2025